In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
import sklearn.metrics as sk
import sys
import os
# Add your local packages directory to Python path
sys.path.append('/eos/home-i02/h/hjia/.local/lib/python3.9/site-packages')
# Print paths to verify
print(sys.path)
import math
phi_res = 128/(2*math.pi)
print(phi_res)
# from dcor import distance_correlation as dcor

2025-05-05 06:45:11.820269: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-05 06:45:14.503037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


['/eos/home-h/hjia/AnomalyDetection/workdir/analysis_notebooks', '/eos/home-h/hjia/miniforge3/envs/ML_env/lib/python311.zip', '/eos/home-h/hjia/miniforge3/envs/ML_env/lib/python3.11', '/eos/home-h/hjia/miniforge3/envs/ML_env/lib/python3.11/lib-dynload', '', '/eos/home-h/hjia/miniforge3/envs/ML_env/lib/python3.11/site-packages', '/eos/home-h/hjia/miniforge3/envs/ML_env/lib/python3.11/site-packages/setuptools/_vendor', '/eos/home-i02/h/hjia/.local/lib/python3.9/site-packages']
20.371832715762604


In [2]:
from tensorflow.keras.layers import BatchNormalization
from qkeras import QActivation, QDense, QDenseBatchnorm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import PReLU, Input, Concatenate, Dense, ReLU, Dropout, BatchNormalization, Activation
from qkeras import quantized_relu, quantized_bits

In [3]:
from matplotlib import cm

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Data
jets = [
    (159.4, 2.45, 0.74),  # pt, eta, phi
    (158.4, 0.95, 4.07),
    (16.6, -2.6, 2.65),
    (15.2, -1.05, 2.70)
]

taus = [
    (48.8, 2.45, 0.74),
    (43.3, 1.05, 4.17),
    (21.6, 0.75, 4.07),
    (8.4, -1.95, 1.82)
]

met_pt, met_phi = 12.1, 1.87

def eta_to_theta(eta):
    return 2 * np.arctan(np.exp(-eta))

def draw_cone(ax, pt, eta, phi, R=0.4):
    # Logarithmic scaling to make smaller jets visible
    scale = (0.3 + 0.7 * np.log10(1 + pt)/np.log10(1 + max(j[0] for j in jets)))*3
    
    t = np.linspace(0, 2*np.pi, 50)
    eta_circle = eta + R * np.cos(t)
    phi_circle = phi + R * np.sin(t)
    
    theta_circle = eta_to_theta(eta_circle)
    x = scale * np.sin(theta_circle) * np.cos(phi_circle)
    y = scale * np.sin(theta_circle) * np.sin(phi_circle)
    z = scale * np.cos(theta_circle)
    
    for i in range(len(t)):
        ax.plot([0, x[i]], [0, y[i]], [0, z[i]], 'b-', alpha=0.3)
    
    ax.plot(x, y, z, 'b-', alpha=0.8)

# Create figure
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot detector cylinder
R = 1
z_points = np.linspace(-4, 4, 100)
phi_points = np.linspace(0, 2*np.pi, 100)
Z, P = np.meshgrid(z_points, phi_points)
X = R * np.cos(P)
Y = R * np.sin(P)
ax.plot_surface(X, Y, Z, alpha=0.1, color='gray')

# Plot jets
for pt, eta, phi in jets:
    draw_cone(ax, pt, eta, phi)

# Plot taus as rays
max_tau_pt = max(t[0] for t in taus)
for pt, eta, phi in taus:
    # Logarithmic scaling for taus
    scale = 0.3 + 0.7 * np.log10(1 + pt)/np.log10(1 + max_tau_pt)
    theta = eta_to_theta(eta)
    x = scale * np.sin(theta) * np.cos(phi)
    y = scale * np.sin(theta) * np.sin(phi)
    z = scale * np.cos(theta)
    ax.plot([0, x], [0, y], [0, z], 'orange', alpha=0.8, linewidth=2)

# Plot MET as ray in x-y plane
# Scale MET separately since it's typically smaller
scale = 0.3 + 0.7 * met_pt/max_tau_pt  # Using tau scale for reference
x = scale * np.cos(met_phi)
y = scale * np.sin(met_phi)
ax.plot([0, x], [0, y], [0, 0], 'r-', alpha=0.8, linewidth=2)

# Labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Most Anomalous Event Display')
ax.view_init(elev=10, azim=-60)
# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='blue', alpha=0.3, label='Jets (ΔR=0.4)'),
    Patch(facecolor='orange', alpha=0.8, label='Taus'),
    Patch(facecolor='red', alpha=0.8, label='MET')
]
ax.legend(handles=legend_elements)

ax.set_box_aspect([1,1,1])

plt.show()

In [ ]:
import numpy as np

jets = [
    (159.4, 2.45, 0.74),  # pt, eta, phi
    (158.4, 0.95, 4.07),
    (16.6, -2.6, 2.65),
    (15.2, -1.05, 2.70)
]

def delta_R(eta1, phi1, eta2, phi2):
    dphi = np.abs(phi1 - phi2)
    dphi = np.where(dphi > np.pi, 2*np.pi - dphi, dphi)
    return np.sqrt((eta1 - eta2)**2 + dphi**2)

def eta_phi_to_xyz(eta, phi):
    theta = 2 * np.arctan(np.exp(-eta))
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return np.array([x, y, z])

# First, let's analyze the actual geometric separation of the jets
print("Jet separations (ΔR):")
for i, (pt1, eta1, phi1) in enumerate(jets):
    for j, (pt2, eta2, phi2) in enumerate(jets[i+1:], i+1):
        dR = delta_R(eta1, phi1, eta2, phi2)
        print(f"Jet {i+1} and {j+1}: ΔR = {dR:.3f}")

# The two high pt jets are:
jet1 = jets[0]  # pt=159.4, eta=2.45, phi=0.74
jet2 = jets[1]  # pt=158.4, eta=0.95, phi=4.07

# Calculate the vector between these two main jets
v1 = eta_phi_to_xyz(jet1[1], jet1[2])
v2 = eta_phi_to_xyz(jet2[1], jet2[2])

# The best view should be perpendicular to both these vectors
view_vector = np.cross(v1, v2)
view_vector = view_vector / np.linalg.norm(view_vector)

# Convert to elevation and azimuth
elevation = np.arcsin(view_vector[2])
azimuth = np.arctan2(view_vector[1], view_vector[0])

# Convert to degrees
best_elev = np.degrees(elevation)
best_azim = np.degrees(azimuth)

print(f"\nBest viewing angles for main jets separation:")
print(f"Elevation: {best_elev:.1f} degrees")
print(f"Azimuth: {best_azim:.1f} degrees")

In [4]:
def has_duplicates(arr):
    _, counts = np.unique(arr, return_counts=True)
    return np.any(counts > 1)

In [5]:
def load_and_preprocess_data(file_path):
    with h5py.File(file_path, 'r') as hf:
        # check if the file is ZeroBias or not
        if "ZB" in file_path:
            print("\nZero Bias data file detected based on filename.")
            ZBflag = 1
        elif "EB" in file_path:
            print("\nEnhanced Bias data file detected based on filename.")
            ZBflag = 0
        else:
            print("\nUnable to determine data type from filename. Please check file naming convention.")
            ZBflag = None 

        # Define object counts
        nmuon, nSRjet, netau = 4, 6, 4

        def fix_phi_range(phi_data):
            # Map values to [0,2pi] range
            phi_fixed = phi_data % (2*math.pi)
            # Scale to [0,128) range and round to nearest integer
            phi_scaled = np.round(phi_fixed * 128/(2*math.pi))
            
            # Take modulo 128 to handle edge cases
            phi_binned = phi_scaled % 128
            return phi_binned
        
        # Load and reshape datasets with scaling
        def load_and_scale(dataset, n_objects, scale_factor=10, eta_factor=40, phi_factor=phi_res):
            data = hf[dataset][:, 0:n_objects, :]
            data[:, :, 0] *= scale_factor  # Scale the pT value
            data[:, :, 1] *= eta_factor  # Scale the angle value
            data[:, :, 2] = fix_phi_range(data[:, :, 2])
            return data.reshape(-1, 3 * n_objects)

        L1_jFexSR_jets = load_and_scale('L1_jFexSR_jets', nSRjet)
        L1_muons = load_and_scale('L1_muons', nmuon, scale_factor=10000)
        L1_eFex_taus = load_and_scale('L1_eFex_taus', netau)

        # Load and process MET
        L1_MET = hf['L1_MET'][:]
        L1_MET[:, 0] *= 10
        L1_MET[:, 2] = fix_phi_range(L1_MET[:, 2])
        
        L1_MET_fixed = np.zeros((L1_MET.shape[0], 2))
        L1_MET_fixed[:, 0] = L1_MET[:, 0]
        L1_MET_fixed[:, 1] = L1_MET[:, 2]
        L1_MET = L1_MET_fixed

        L1_mu = hf['mu'][:]
        valid_events_mask = L1_mu != 0
        
        EB_weights = hf["EB_weights"][:]
        if ZBflag == 0:
            L1_LBmu = np.ones(len(hf['mu'][:]))*61
        elif ZBflag == 1:
            L1_LBmu = hf['averageInteractionsPerCrossing'][:]
        weighted_mu_zero_fraction = np.sum(EB_weights[L1_mu == 0]) / np.sum(EB_weights)
        print(f"Weighted fraction of events with mu=0: {weighted_mu_zero_fraction:.4f}")

        pass_L1_unprescaled = hf["pass_L1_unprescaled"][:]
                
        # Combine arrays into Topo 2A
        Topo_2A = np.concatenate([L1_jFexSR_jets, L1_eFex_taus, L1_muons, L1_MET], axis=1)
        print("Shape of Topo_2A:", Topo_2A.shape)

        Topo_2A = Topo_2A[valid_events_mask]
        pass_L1_unprescaled = pass_L1_unprescaled[valid_events_mask]
        L1_mu = L1_mu[valid_events_mask]
        EB_weights = EB_weights[valid_events_mask]
        L1_LBmu = L1_LBmu[valid_events_mask]
        return Topo_2A, pass_L1_unprescaled, L1_mu, EB_weights, L1_LBmu

In [6]:
def load_and_preprocess_normal_multiple_files(file_paths):
    # Initialize lists to store data from each file
    all_Topo_2A = []
    all_pass_L1 = []
    all_mu = []
    all_weights = []
    all_LBmu = []
    
    # Load data from each file
    for file_path in file_paths:
        Topo_2A, pass_L1_unprescaled, L1_mu, L1_weights, L1_LBmu = load_and_preprocess_data(file_path)
        if np.sum(pass_L1_unprescaled) == 0:
            print("no L1 pass events!!!")
        all_Topo_2A.append(Topo_2A)
        all_pass_L1.append(pass_L1_unprescaled)
        all_mu.append(L1_mu)
        all_weights.append(L1_weights)
        all_LBmu.append(L1_LBmu)
    
    # Concatenate all files' data
    final_Topo_2A = np.concatenate(all_Topo_2A, axis=0)
    final_pass_L1 = np.concatenate(all_pass_L1, axis=0)
    final_mu = np.concatenate(all_mu, axis=0)
    final_weights = np.concatenate(all_weights, axis=0)
    final_LBmu = np.concatenate(all_LBmu, axis=0)
    # Get total number of samples
    n_samples = final_Topo_2A.shape[0]
    
    # Set random seed and create shuffle indices
    np.random.seed(42)
    shuffle_idx = np.random.permutation(n_samples)
    
    # Shuffle all arrays using the same indices
    final_Topo_2A = final_Topo_2A[shuffle_idx]
    final_pass_L1 = final_pass_L1[shuffle_idx]
    final_mu = final_mu[shuffle_idx]
    final_weights = final_weights[shuffle_idx]
    final_LBmu = final_LBmu[shuffle_idx]

    def fill_median(array):
        for i in range(array.shape[1]):
            array[np.isnan(array[:, i]), i] = 0#median_value
        return array

    final_Topo_2A = fill_median(final_Topo_2A)

    print(f"total of {n_samples} event processed")
    return final_Topo_2A, final_pass_L1, final_mu, final_weights, final_LBmu


In [7]:
file_paths = [
    '/eos/home-m/mmcohen/ntuples/04-15-2025/EB/EB_473255_0_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/EB/EB_475321_0_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/EB/EB_482596_0_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_474448_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_474462_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_474562_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/misc/ZB_474657.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_474679_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_474926_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_474991_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_475008_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_475052_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_475066_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_475168_04-15-2025.h5',
    '/eos/home-m/mmcohen/ntuples/04-15-2025/ZB/ZB_zeroBias_475192_04-15-2025.h5',
]    

In [8]:
Topo_2A, pass_L1_unprescaled, event_mu, events_weights, event_LBmu = load_and_preprocess_normal_multiple_files(file_paths)


Enhanced Bias data file detected based on filename.
Weighted fraction of events with mu=0: 0.7770
Shape of Topo_2A: (1146112, 44)

Enhanced Bias data file detected based on filename.
Weighted fraction of events with mu=0: 0.1958
Shape of Topo_2A: (1119657, 44)

Enhanced Bias data file detected based on filename.
Weighted fraction of events with mu=0: 0.1952
Shape of Topo_2A: (1048336, 44)

Zero Bias data file detected based on filename.
Weighted fraction of events with mu=0: 0.0000
Shape of Topo_2A: (152922, 44)

Zero Bias data file detected based on filename.
Weighted fraction of events with mu=0: 0.0000
Shape of Topo_2A: (262296, 44)

Zero Bias data file detected based on filename.
Weighted fraction of events with mu=0: 0.0000
Shape of Topo_2A: (442132, 44)

Zero Bias data file detected based on filename.
Weighted fraction of events with mu=0: 0.0000
Shape of Topo_2A: (392834, 44)

Zero Bias data file detected based on filename.
Weighted fraction of events with mu=0: 0.0000
Shape of

In [13]:
print(Topo_2A.shape)

(6397402, 44)


In [9]:
Topo_2A_test = Topo_2A[3000000:,:]
L1_pass_flag = pass_L1_unprescaled[3000000:]
Topo_2A_mu = event_mu[3000000:]
Topo_test_weights = events_weights[3000000:]
Topo_2A_LBmu = event_LBmu[3000000:]

In [15]:
print(Topo_2A_test.shape)

(3397402, 44)


In [10]:
Topo_2A_test = Topo_2A_test[Topo_2A_LBmu >= 60]
L1_pass_flag = L1_pass_flag[Topo_2A_LBmu >= 60]
Topo_2A_mu = Topo_2A_mu[Topo_2A_LBmu >= 60]
Topo_test_weights = Topo_test_weights[Topo_2A_LBmu >= 60]
Topo_2A_LBmu = Topo_2A_LBmu[Topo_2A_LBmu >= 60]

In [11]:
Topo_2A_test = Topo_2A_test[Topo_2A_mu >= 52]
L1_pass_flag = L1_pass_flag[Topo_2A_mu >= 52]
Topo_test_weights = Topo_test_weights[Topo_2A_mu >= 52]
Topo_2A_LBmu = Topo_2A_LBmu[Topo_2A_mu >= 52]
Topo_2A_mu = Topo_2A_mu[Topo_2A_mu >= 52]

In [ ]:
print(Topo_2A_test.shape)

In [ ]:
print(np.sum(L1_pass_flag))

In [12]:
def load_and_process_anomalous_data(file_name):
    with h5py.File(file_name, 'r') as hf:
        nmuon, nLRjet, nSRjet, negamma, netau, njtau = 4, 6, 6, 4, 4, 4
        print(hf.keys())
        def fix_phi_range(phi_data):
            # Map values to [0,2pi] range
            phi_fixed = phi_data % (2*math.pi)
            # Scale to [0,128) range and round to nearest integer
            phi_scaled = np.round(phi_fixed * 128/(2*math.pi))
            
            # Take modulo 128 to handle edge cases
            phi_binned = phi_scaled % 128
            return phi_binned

        def load_and_scale(dataset, n_objects, scale_factor=10, eta_factor=40):
            data = hf[dataset][:, 0:n_objects, :]
            data[:, :, 0] *= scale_factor  # Scale the pT value
            data[:, :, 1] *= eta_factor  # Scale the angle value
            data[:, :, 2] = fix_phi_range(data[:, :, 2])
            return data.reshape(-1, 3 * n_objects)

        L1_jFexSR_jets = load_and_scale('L1_jFexSR_jets', nSRjet)
        L1_jFexLR_jets = load_and_scale('L1_jFexLR_jets', nLRjet)
        L1_egammas = load_and_scale('L1_egammas', negamma)
        L1_muons = load_and_scale('L1_muons', nmuon, scale_factor=10000)  # Specific scaling for muons
        L1_eFex_taus = load_and_scale('L1_eFex_taus', netau)
        L1_jFex_taus = load_and_scale('L1_jFex_taus', njtau)

        L1_MET = hf['L1_MET'][:]
        L1_MET[:, 0] *= 10
        L1_MET[:, 2] = fix_phi_range(L1_MET[:, 2])

        pass_L1_unprescaled = hf["pass_L1_unprescaled"][:]

        # Reformat L1_MET
        L1_MET_fixed = np.zeros((L1_MET.shape[0], 2))
        L1_MET_fixed[:, 0] = L1_MET[:, 0]
        L1_MET_fixed[:, 1] = L1_MET[:, 2]
        L1_MET = L1_MET_fixed

        # Combine arrays into Topo groups
        Topo_2A = np.concatenate([L1_jFexSR_jets, L1_eFex_taus, L1_muons, L1_MET], axis=1)

        # Handle NaN values
        def fill_median(array):
            for i in range(array.shape[1]):
                median_value = np.nanmedian(array[:, i])
                array[np.isnan(array[:, i]), i] = 0#median_value
            return array

        Topo_2A = fill_median(Topo_2A)

        return Topo_2A, pass_L1_unprescaled

In [39]:
# Topo_2A_HHbbtt, pass_L1_HHbbtt = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/HHbbttHadHad_07-10-2024.h5')
# Topo_2A_A14, pass_L1_A14 = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/A14N23LO_07-17-2024.h5')
# Topo_2A_HAHMggf, pass_L1_HAHMggf = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/HAHMggfZdZd2l2nu_07-17-2024.h5')
# Topo_2A_qqa, pass_L1_qqa = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/qqa_07-17-2024.h5')
# Topo_2A_Zprime, pass_L1_Zprime = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/Zprime2EJs_07-17-2024.h5')
# Topo_2A_ZZ4lep, pass_L1_ZZ4lep = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/ZZ4lep_07-17-2024.h5')
# Topo_2A_jz1, pass_L1_jz1 = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/jjJZ1_07-17-2024.h5')
# Topo_2A_jz2, pass_L1_jz2 = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/jjJZ2_07-17-2024.h5')
# Topo_2A_jz4, pass_L1_jz4 = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/MC_07-17-2024/jjJZ4_07-17-2024.h5')

In [13]:
Topo_2A_ChiPlusChiMinus100_99_0p3ns, pass_L1_ChiPlusChiMinus100_99_0p3ns = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_ChiPlusChiMinus100_99_0p3ns_04-15-2025.h5')
Topo_2A_ChiPlusChiMinus500_40_10ns, pass_L1_ChiPlusChiMinus500_40_10ns = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_ChiPlusChiMinus500_40_10ns_04-15-2025.h5')
Topo_2A_HAHMggf_23e, pass_L1_HAHMggf_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_HAHMggf_04-15-2025.h5')
Topo_2A_HNLeemu_23e, pass_L1_HNLeemu_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_HNLeemu_04-15-2025.h5')
Topo_2A_jz2_23e, pass_L1_jz2_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_jjJZ2_04-15-2025.h5')
Topo_2A_jz4_23e, pass_L1_jz4_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_jjJZ4_04-15-2025.h5')
Topo_2A_HH4b_23e, pass_L1_HH4b_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_RS_G_hh_bbbb_AutoWidth_c10_M300_04-15-2025.h5')
Topo_2A_ttbar_1lep_23e, pass_L1_ttbar_1lep_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/misc/mc23e_ttbar_1lep.h5')
Topo_2A_ttbar_2lep_23e, pass_L1_ttbar_2lep_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_ttbar_2lep_04-15-2025.h5')
Topo_2A_ZZ4lep_23e, pass_L1_ZZ4lep_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC/MC_ZZ4lep_04-15-2025.h5')

<KeysViewHDF5 ['EB_weights', 'HLT_MET_cell', 'HLT_MET_mhtpufit_pf', 'HLT_MET_mhtpufit_pf_subjesgscIS', 'HLT_MET_nn', 'HLT_MET_pfopufit', 'HLT_MET_tcpufit', 'HLT_electrons', 'HLT_jets', 'HLT_muons', 'HLT_photons', 'L1_MET', 'L1_eFex_taus', 'L1_egammas', 'L1_jFexLR_jets', 'L1_jFexSR_jets', 'L1_jFex_taus', 'L1_muons', 'LRT_electrons', 'LRT_muons', 'averageInteractionsPerCrossing', 'event_number', 'lumiBlock', 'lumiblock_bads', 'mu', 'pass_HLT_unprescaled', 'pass_L1_unprescaled', 'run_number']>
<KeysViewHDF5 ['EB_weights', 'HLT_MET_cell', 'HLT_MET_mhtpufit_pf', 'HLT_MET_mhtpufit_pf_subjesgscIS', 'HLT_MET_nn', 'HLT_MET_pfopufit', 'HLT_MET_tcpufit', 'HLT_electrons', 'HLT_jets', 'HLT_muons', 'HLT_photons', 'L1_MET', 'L1_eFex_taus', 'L1_egammas', 'L1_jFexLR_jets', 'L1_jFexSR_jets', 'L1_jFex_taus', 'L1_muons', 'LRT_electrons', 'LRT_muons', 'averageInteractionsPerCrossing', 'event_number', 'lumiBlock', 'lumiblock_bads', 'mu', 'pass_HLT_unprescaled', 'pass_L1_unprescaled', 'run_number']>
<KeysVie

In [14]:
Topo_2A_CC_directHlR23_150_23e, pass_L1_CC_directHlR23_150_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_CC_directHlR23_150_04-15-2025.h5')
Topo_2A_HtoSUEP_VBF_23e, pass_L1_HtoSUEP_VBF_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_HtoSUEP_VBF_fullhad_125_3p00_4p00_VBFFilter_04-15-2025.h5')
Topo_2A_ggF_H125_a16a16_23e, pass_L1_ggF_H125_a16a16_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_ggF_H125_a16a16_4b_ctau10_filtered_04-15-2025.h5')
Topo_2A_nnlo30_ggH_2a12_23e, pass_L1_nnlo30_ggH_2a12_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_nnlo30_A14_ggH_H125_2a12_2b2mu_04-15-2025.h5')
Topo_2A_ggF_H125_a55a55_100_23e, pass_L1_ggF_H125_a55a55_100_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_ggF_H125_a55a55_4b_ctau100_filtered_04-15-2025.h5')
Topo_2A_NNLO_QED_EE_yytautau_23e, pass_L1_NNLO_QED_EE_yytautau_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_NNLO_QED_EE_yytautau_04-15-2025.h5')
Topo_2A_ggF_H125_a55a55_10_23e, pass_L1_ggF_H125_a55a55_10_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_ggF_H125_a55a55_4b_ctau10_filtered_04-15-2025.h5')
Topo_2A_qqa_Ph25_mRp125_23e, pass_L1_qqa_Ph25_mRp125_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_qqa_Ph25_mRp125_gASp1_qContentB_04-15-2025.h5')
Topo_2A_qqa_Ph25_mRp150_23e, pass_L1_qqa_Ph25_mRp150_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_qqa_Ph25_mRp150_gASp1_qContentUDSC_04-15-2025.h5')
Topo_2A_qqa_Ph25_mRp300_23e, pass_L1_qqa_Ph25_mRp300_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_qqa_Ph25_mRp300_gASp1_qContentUDCS_04-15-2025.h5')
Topo_2A_HAHM_S2Zd4e_23e, pass_L1_HAHM_S2Zd4e_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_HAHM_S2Zd4e_60_25_0p1ns_04-15-2025.h5')
Topo_2A_HAHM_S2Zd4mu_23e, pass_L1_HAHM_S2Zd4mu_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_HAHM_S2Zd4mu_60_25_0p1ns_04-15-2025.h5')
Topo_2A_RS_G_hh_M1000_23e, pass_L1_RS_G_hh_M1000_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_RS_G_hh_bbbb_AutoWidth_c10_M1000_04-15-2025.h5')
Topo_2A_hh_vbf_l0cvv1cv1_23e, pass_L1_hh_vbf_l0cvv1cv1_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_hh_bbbb_vbf_novhh_5fs_l0cvv1cv1_04-15-2025.h5')
Topo_2A_RS_G_hh_M600_23e, pass_L1_RS_G_hh_M600_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_RS_G_hh_bbbb_AutoWidth_c10_M600_04-15-2025.h5')
Topo_2A_VBF_H125_a55a55_1_23e, pass_L1_VBF_H125_a55a55_1_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_VBF_H125_a55a55_4b_ctau1_filtered_04-15-2025.h5')
Topo_2A_ZllH_H125_ctau001_23e, pass_L1_ZllH_H125_ctau001_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_ZllH_H125_a5a5_4g_ctau001_04-15-2025.h5')
Topo_2A_ZllH_H125_ctau10_23e, pass_L1_ZllH_H125_ctau10_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_ZllH_H125_a5a5_4g_ctau10_04-15-2025.h5')
Topo_2A_Znunu_BFilter_23e, pass_L1_Znunu_BFilter_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_Znunu_FxFx3jHT2bias_SW_pTvv70_BFilter_04-15-2025.h5')
Topo_2A_Znunu_CFilterBVeto_23e, pass_L1_Znunu_CFilterBVeto_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_Znunu_FxFx3jHT2bias_SW_pTvv70_CFilterBVeto_04-15-2025.h5')
Topo_2A_Zp_ee_ZeroWidth3000_23e, pass_L1_Zp_ee_ZeroWidth3000_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_Zp_ee_ZeroWidth3000_04-15-2025.h5')
Topo_2A_HtoSUEP_ggH_125_23e, pass_L1_HtoSUEP_ggH_125_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_HtoSUEP_ggH_fullhad_125_3p00_3p00_noFilter_04-15-2025.h5')
Topo_2A_HtoSUEP_ggH_500_23e, pass_L1_HtoSUEP_ggH_500_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_HtoSUEP_ggH_fullhad_500_3p00_3p00_noFilter_04-15-2025.h5')
Topo_2A_hh_vbf_l10cvv1cv1_23e, pass_L1_hh_vbf_l10cvv1cv1_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_hh_bbbb_vbf_novhh_5fs_l10cvv1cv1_04-15-2025.h5')
Topo_2A_hh_vbf_l1cvv0cv1_23e, pass_L1_hh_vbf_l1cvv0cv1_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_hh_bbbb_vbf_novhh_5fs_l1cvv0cv1_04-15-2025.h5')
Topo_2A_hh_vbf_l1cvv1cv1_23e, pass_L1_hh_vbf_l1cvv1cv1_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_hh_bbbb_vbf_novhh_5fs_l1cvv1cv1_04-15-2025.h5')
Topo_2A_hh_vbf_l1cvv2cv1_23e, pass_L1_hh_vbf_l1cvv2cv1_23e = load_and_process_anomalous_data('/eos/home-m/mmcohen/ntuples/04-15-2025/MC_new/MC_hh_bbbb_vbf_novhh_5fs_l1cvv2cv1_04-15-2025.h5')

<KeysViewHDF5 ['EB_weights', 'HLT_MET_cell', 'HLT_MET_mhtpufit_pf', 'HLT_MET_mhtpufit_pf_subjesgscIS', 'HLT_MET_nn', 'HLT_MET_pfopufit', 'HLT_MET_tcpufit', 'HLT_electrons', 'HLT_jets', 'HLT_muons', 'HLT_photons', 'L1_MET', 'L1_eFex_taus', 'L1_egammas', 'L1_jFexLR_jets', 'L1_jFexSR_jets', 'L1_jFex_taus', 'L1_muons', 'LRT_electrons', 'LRT_muons', 'averageInteractionsPerCrossing', 'event_number', 'lumiBlock', 'lumiblock_bads', 'mu', 'pass_HLT_unprescaled', 'pass_L1_unprescaled', 'run_number']>
<KeysViewHDF5 ['EB_weights', 'HLT_MET_cell', 'HLT_MET_mhtpufit_pf', 'HLT_MET_mhtpufit_pf_subjesgscIS', 'HLT_MET_nn', 'HLT_MET_pfopufit', 'HLT_MET_tcpufit', 'HLT_electrons', 'HLT_jets', 'HLT_muons', 'HLT_photons', 'L1_MET', 'L1_eFex_taus', 'L1_egammas', 'L1_jFexLR_jets', 'L1_jFexSR_jets', 'L1_jFex_taus', 'L1_muons', 'LRT_electrons', 'LRT_muons', 'averageInteractionsPerCrossing', 'event_number', 'lumiBlock', 'lumiblock_bads', 'mu', 'pass_HLT_unprescaled', 'pass_L1_unprescaled', 'run_number']>
<KeysVie

In [15]:
#mu_veto_mask = Topo_2A_mu != 0
L1_veto_mask = L1_pass_flag == 0
combined_mask = L1_veto_mask# & mu_veto_mask
#print(mu_veto_mask.shape)
print(L1_veto_mask.shape)
print(combined_mask.shape)
print(Topo_test_weights.shape)
Topo_2A_L1failed = Topo_2A_test[combined_mask]
Topo_2A_L1failed_mu = Topo_2A_mu[combined_mask]
Topo_EB_L1failed_weights = Topo_test_weights[combined_mask]

weights_selected = Topo_test_weights[combined_mask]
print(np.sum(weights_selected) / np.sum(Topo_test_weights))

(1912367,)
(1912367,)
(1912367,)


0.9966735071934902


In [16]:
Filter_eff = np.sum(weights_selected) / np.sum(Topo_test_weights)
print(Filter_eff)

0.9966735071934902


In [44]:
print(Topo_2A_L1failed.shape)

(1369041, 44)


In [17]:
# Topo_2A_HHbbtt_pure = Topo_2A_HHbbtt[pass_L1_HHbbtt == 0]
# Topo_2A_A14_pure = Topo_2A_A14[pass_L1_A14 == 0]
# Topo_2A_HAHMggf_pure = Topo_2A_HAHMggf[pass_L1_HAHMggf == 0]
# Topo_2A_qqa_pure = Topo_2A_qqa[pass_L1_qqa == 0]
# Topo_2A_Zprime_pure = Topo_2A_Zprime[pass_L1_Zprime == 0]
# Topo_2A_ZZ4lep_pure = Topo_2A_ZZ4lep[pass_L1_ZZ4lep == 0]
# Topo_2A_jz1_pure = Topo_2A_jz1[pass_L1_jz1 == 0]
# Topo_2A_jz2_pure = Topo_2A_jz2[pass_L1_jz2 == 0]
# Topo_2A_jz4_pure = Topo_2A_jz4[pass_L1_jz4 == 0]

# 23e
Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure = Topo_2A_ChiPlusChiMinus100_99_0p3ns[pass_L1_ChiPlusChiMinus100_99_0p3ns == 0]
Topo_2A_ChiPlusChiMinus500_40_10ns_pure = Topo_2A_ChiPlusChiMinus500_40_10ns[pass_L1_ChiPlusChiMinus500_40_10ns == 0]
Topo_2A_HAHMggf_23e_pure = Topo_2A_HAHMggf_23e[pass_L1_HAHMggf_23e == 0]
Topo_2A_HNLeemu_23e_pure = Topo_2A_HNLeemu_23e[pass_L1_HNLeemu_23e == 0]
Topo_2A_jz2_23e_pure = Topo_2A_jz2_23e[pass_L1_jz2_23e == 0]
Topo_2A_jz4_23e_pure = Topo_2A_jz4_23e[pass_L1_jz4_23e == 0]
Topo_2A_HH4b_23e_pure = Topo_2A_HH4b_23e[pass_L1_HH4b_23e == 0]
Topo_2A_ttbar_1lep_23e_pure = Topo_2A_ttbar_1lep_23e[pass_L1_ttbar_1lep_23e == 0]
Topo_2A_ttbar_2lep_23e_pure = Topo_2A_ttbar_2lep_23e[pass_L1_ttbar_2lep_23e == 0]
Topo_2A_ZZ4lep_23e_pure = Topo_2A_ZZ4lep_23e[pass_L1_ZZ4lep_23e == 0]

In [18]:
Topo_2A_CC_directHlR23_150_23e_pure = Topo_2A_CC_directHlR23_150_23e[pass_L1_CC_directHlR23_150_23e == 0]
Topo_2A_HtoSUEP_VBF_23e_pure = Topo_2A_HtoSUEP_VBF_23e[pass_L1_HtoSUEP_VBF_23e == 0]
Topo_2A_ggF_H125_a16a16_23e_pure = Topo_2A_ggF_H125_a16a16_23e[pass_L1_ggF_H125_a16a16_23e == 0]
Topo_2A_nnlo30_ggH_2a12_23e_pure = Topo_2A_nnlo30_ggH_2a12_23e[pass_L1_nnlo30_ggH_2a12_23e == 0]
Topo_2A_ggF_H125_a55a55_100_23e_pure = Topo_2A_ggF_H125_a55a55_100_23e[pass_L1_ggF_H125_a55a55_100_23e == 0]
Topo_2A_NNLO_QED_EE_yytautau_23e_pure = Topo_2A_NNLO_QED_EE_yytautau_23e[pass_L1_NNLO_QED_EE_yytautau_23e == 0]
Topo_2A_ggF_H125_a55a55_10_23e_pure = Topo_2A_ggF_H125_a55a55_10_23e[pass_L1_ggF_H125_a55a55_10_23e == 0]
Topo_2A_qqa_Ph25_mRp125_23e_pure = Topo_2A_qqa_Ph25_mRp125_23e[pass_L1_qqa_Ph25_mRp125_23e == 0]
Topo_2A_qqa_Ph25_mRp150_23e_pure = Topo_2A_qqa_Ph25_mRp150_23e[pass_L1_qqa_Ph25_mRp150_23e == 0]
Topo_2A_qqa_Ph25_mRp300_23e_pure = Topo_2A_qqa_Ph25_mRp300_23e[pass_L1_qqa_Ph25_mRp300_23e == 0]
Topo_2A_HAHM_S2Zd4e_23e_pure = Topo_2A_HAHM_S2Zd4e_23e[pass_L1_HAHM_S2Zd4e_23e == 0]
Topo_2A_HAHM_S2Zd4mu_23e_pure = Topo_2A_HAHM_S2Zd4mu_23e[pass_L1_HAHM_S2Zd4mu_23e == 0]
Topo_2A_RS_G_hh_M1000_23e_pure = Topo_2A_RS_G_hh_M1000_23e[pass_L1_RS_G_hh_M1000_23e == 0]
Topo_2A_hh_vbf_l0cvv1cv1_23e_pure = Topo_2A_hh_vbf_l0cvv1cv1_23e[pass_L1_hh_vbf_l0cvv1cv1_23e == 0]
Topo_2A_RS_G_hh_M600_23e_pure = Topo_2A_RS_G_hh_M600_23e[pass_L1_RS_G_hh_M600_23e == 0]
Topo_2A_VBF_H125_a55a55_1_23e_pure = Topo_2A_VBF_H125_a55a55_1_23e[pass_L1_VBF_H125_a55a55_1_23e == 0]
Topo_2A_ZllH_H125_ctau001_23e_pure = Topo_2A_ZllH_H125_ctau001_23e[pass_L1_ZllH_H125_ctau001_23e == 0]
Topo_2A_ZllH_H125_ctau10_23e_pure = Topo_2A_ZllH_H125_ctau10_23e[pass_L1_ZllH_H125_ctau10_23e == 0]
Topo_2A_Znunu_BFilter_23e_pure = Topo_2A_Znunu_BFilter_23e[pass_L1_Znunu_BFilter_23e == 0]
Topo_2A_Znunu_CFilterBVeto_23e_pure = Topo_2A_Znunu_CFilterBVeto_23e[pass_L1_Znunu_CFilterBVeto_23e == 0]
Topo_2A_Zp_ee_ZeroWidth3000_23e_pure = Topo_2A_Zp_ee_ZeroWidth3000_23e[pass_L1_Zp_ee_ZeroWidth3000_23e == 0]
Topo_2A_HtoSUEP_ggH_125_23e_pure = Topo_2A_HtoSUEP_ggH_125_23e[pass_L1_HtoSUEP_ggH_125_23e == 0]
Topo_2A_HtoSUEP_ggH_500_23e_pure = Topo_2A_HtoSUEP_ggH_500_23e[pass_L1_HtoSUEP_ggH_500_23e == 0]
Topo_2A_hh_vbf_l10cvv1cv1_23e_pure = Topo_2A_hh_vbf_l10cvv1cv1_23e[pass_L1_hh_vbf_l10cvv1cv1_23e == 0]
Topo_2A_hh_vbf_l1cvv0cv1_23e_pure = Topo_2A_hh_vbf_l1cvv0cv1_23e[pass_L1_hh_vbf_l1cvv0cv1_23e == 0]
Topo_2A_hh_vbf_l1cvv1cv1_23e_pure = Topo_2A_hh_vbf_l1cvv1cv1_23e[pass_L1_hh_vbf_l1cvv1cv1_23e == 0]
Topo_2A_hh_vbf_l1cvv2cv1_23e_pure = Topo_2A_hh_vbf_l1cvv2cv1_23e[pass_L1_hh_vbf_l1cvv2cv1_23e == 0]

In [19]:
def calculate_percentage(original, filtered):
    return (len(filtered) / len(original)) * 100 if len(original) > 0 else 0

datasets = [
    # ("Topo_2A_HHbbtt", Topo_2A_HHbbtt, Topo_2A_HHbbtt_pure),
    # ("Topo_2A_A14", Topo_2A_A14, Topo_2A_A14_pure),
    # ("Topo_2A_HAHMggf", Topo_2A_HAHMggf, Topo_2A_HAHMggf_pure),
    # ("Topo_2A_qqa", Topo_2A_qqa, Topo_2A_qqa_pure),
    # ("Topo_2A_Zprime", Topo_2A_Zprime, Topo_2A_Zprime_pure),
    # ("Topo_2A_ZZ4lep", Topo_2A_ZZ4lep, Topo_2A_ZZ4lep_pure),
    # ("Topo_2A_jz1", Topo_2A_jz1, Topo_2A_jz1_pure),
    # ("Topo_2A_jz2", Topo_2A_jz2, Topo_2A_jz2_pure),
    # ("Topo_2A_jz4", Topo_2A_jz4, Topo_2A_jz4_pure),
    ("Topo_2A_ChiPlusChiMinus100_99_0p3ns", Topo_2A_ChiPlusChiMinus100_99_0p3ns, Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure),
    ("Topo_2A_ChiPlusChiMinus500_40_10ns", Topo_2A_ChiPlusChiMinus500_40_10ns, Topo_2A_ChiPlusChiMinus500_40_10ns_pure),
    ("Topo_2A_HAHMggf_23e", Topo_2A_HAHMggf_23e, Topo_2A_HAHMggf_23e_pure),
    ("Topo_2A_HNLeemu_23e", Topo_2A_HNLeemu_23e, Topo_2A_HNLeemu_23e_pure),
    ("Topo_2A_jz2_23e", Topo_2A_jz2_23e, Topo_2A_jz2_23e_pure),
    ("Topo_2A_jz4_23e", Topo_2A_jz4_23e, Topo_2A_jz4_23e_pure),
    ("Topo_2A_HH4b_23e", Topo_2A_HH4b_23e, Topo_2A_HH4b_23e_pure),
    ("Topo_2A_ttbar_1lep_23e", Topo_2A_ttbar_1lep_23e, Topo_2A_ttbar_1lep_23e_pure),
    ("Topo_2A_ttbar_2lep_23e", Topo_2A_ttbar_2lep_23e, Topo_2A_ttbar_2lep_23e_pure),
    ("Topo_2A_ZZ4lep_23e", Topo_2A_ZZ4lep_23e, Topo_2A_ZZ4lep_23e_pure),
    ("Topo_2A_CC_directHlR23_150_23e", Topo_2A_CC_directHlR23_150_23e, Topo_2A_CC_directHlR23_150_23e_pure),
    ("Topo_2A_HtoSUEP_VBF_23e", Topo_2A_HtoSUEP_VBF_23e, Topo_2A_HtoSUEP_VBF_23e_pure),
    ("Topo_2A_ggF_H125_a16a16_23e", Topo_2A_ggF_H125_a16a16_23e, Topo_2A_ggF_H125_a16a16_23e_pure),
    ("Topo_2A_nnlo30_ggH_2a12_23e", Topo_2A_nnlo30_ggH_2a12_23e, Topo_2A_nnlo30_ggH_2a12_23e_pure),
    ("Topo_2A_ggF_H125_a55a55_100_23e", Topo_2A_ggF_H125_a55a55_100_23e, Topo_2A_ggF_H125_a55a55_100_23e_pure),
    ("Topo_2A_NNLO_QED_EE_yytautau_23e", Topo_2A_NNLO_QED_EE_yytautau_23e, Topo_2A_NNLO_QED_EE_yytautau_23e_pure),
    ("Topo_2A_ggF_H125_a55a55_10_23e", Topo_2A_ggF_H125_a55a55_10_23e, Topo_2A_ggF_H125_a55a55_10_23e_pure),
    ("Topo_2A_qqa_Ph25_mRp125_23e", Topo_2A_qqa_Ph25_mRp125_23e, Topo_2A_qqa_Ph25_mRp125_23e_pure),
    ("Topo_2A_qqa_Ph25_mRp150_23e", Topo_2A_qqa_Ph25_mRp150_23e, Topo_2A_qqa_Ph25_mRp150_23e_pure),
    ("Topo_2A_qqa_Ph25_mRp300_23e", Topo_2A_qqa_Ph25_mRp300_23e, Topo_2A_qqa_Ph25_mRp300_23e_pure),
    ("Topo_2A_HAHM_S2Zd4e_23e", Topo_2A_HAHM_S2Zd4e_23e, Topo_2A_HAHM_S2Zd4e_23e_pure),
    ("Topo_2A_HAHM_S2Zd4mu_23e", Topo_2A_HAHM_S2Zd4mu_23e, Topo_2A_HAHM_S2Zd4mu_23e_pure),
    ("Topo_2A_RS_G_hh_M1000_23e", Topo_2A_RS_G_hh_M1000_23e, Topo_2A_RS_G_hh_M1000_23e_pure),
    ("Topo_2A_hh_vbf_l0cvv1cv1_23e", Topo_2A_hh_vbf_l0cvv1cv1_23e, Topo_2A_hh_vbf_l0cvv1cv1_23e_pure),
    ("Topo_2A_RS_G_hh_M600_23e", Topo_2A_RS_G_hh_M600_23e, Topo_2A_RS_G_hh_M600_23e_pure),
    ("Topo_2A_VBF_H125_a55a55_1_23e", Topo_2A_VBF_H125_a55a55_1_23e, Topo_2A_VBF_H125_a55a55_1_23e_pure),
    ("Topo_2A_ZllH_H125_ctau001_23e", Topo_2A_ZllH_H125_ctau001_23e, Topo_2A_ZllH_H125_ctau001_23e_pure),
    ("Topo_2A_ZllH_H125_ctau10_23e", Topo_2A_ZllH_H125_ctau10_23e, Topo_2A_ZllH_H125_ctau10_23e_pure),
    ("Topo_2A_Znunu_BFilter_23e", Topo_2A_Znunu_BFilter_23e, Topo_2A_Znunu_BFilter_23e_pure),
    ("Topo_2A_Znunu_CFilterBVeto_23e", Topo_2A_Znunu_CFilterBVeto_23e, Topo_2A_Znunu_CFilterBVeto_23e_pure),
    ("Topo_2A_Zp_ee_ZeroWidth3000_23e", Topo_2A_Zp_ee_ZeroWidth3000_23e, Topo_2A_Zp_ee_ZeroWidth3000_23e_pure),
    ("Topo_2A_HtoSUEP_ggH_125_23e", Topo_2A_HtoSUEP_ggH_125_23e, Topo_2A_HtoSUEP_ggH_125_23e_pure),
    ("Topo_2A_HtoSUEP_ggH_500_23e", Topo_2A_HtoSUEP_ggH_500_23e, Topo_2A_HtoSUEP_ggH_500_23e_pure),
    ("Topo_2A_hh_vbf_l10cvv1cv1_23e", Topo_2A_hh_vbf_l10cvv1cv1_23e, Topo_2A_hh_vbf_l10cvv1cv1_23e_pure),
    ("Topo_2A_hh_vbf_l1cvv0cv1_23e", Topo_2A_hh_vbf_l1cvv0cv1_23e, Topo_2A_hh_vbf_l1cvv0cv1_23e_pure),
    ("Topo_2A_hh_vbf_l1cvv1cv1_23e", Topo_2A_hh_vbf_l1cvv1cv1_23e, Topo_2A_hh_vbf_l1cvv1cv1_23e_pure),
    ("Topo_2A_hh_vbf_l1cvv2cv1_23e", Topo_2A_hh_vbf_l1cvv2cv1_23e, Topo_2A_hh_vbf_l1cvv2cv1_23e_pure),
]

# Dictionary to store percentages
filter_percentages = {}

for name, original, filtered in datasets:
    percentage = calculate_percentage(original, filtered)
    filter_percentages[name] = percentage
    print(f"{name}: {percentage:.2f}% passed the filter")

Topo_2A_ChiPlusChiMinus100_99_0p3ns: 19.95% passed the filter
Topo_2A_ChiPlusChiMinus500_40_10ns: 47.56% passed the filter
Topo_2A_HAHMggf_23e: 16.69% passed the filter
Topo_2A_HNLeemu_23e: 43.32% passed the filter
Topo_2A_jz2_23e: 52.34% passed the filter
Topo_2A_jz4_23e: 0.00% passed the filter
Topo_2A_HH4b_23e: 13.16% passed the filter
Topo_2A_ttbar_1lep_23e: 5.00% passed the filter
Topo_2A_ttbar_2lep_23e: 4.97% passed the filter
Topo_2A_ZZ4lep_23e: 5.88% passed the filter
Topo_2A_CC_directHlR23_150_23e: 8.33% passed the filter
Topo_2A_HtoSUEP_VBF_23e: 31.97% passed the filter
Topo_2A_ggF_H125_a16a16_23e: 8.69% passed the filter
Topo_2A_nnlo30_ggH_2a12_23e: 17.43% passed the filter
Topo_2A_ggF_H125_a55a55_100_23e: 5.59% passed the filter
Topo_2A_NNLO_QED_EE_yytautau_23e: 79.09% passed the filter
Topo_2A_ggF_H125_a55a55_10_23e: 5.82% passed the filter
Topo_2A_qqa_Ph25_mRp125_23e: 5.52% passed the filter
Topo_2A_qqa_Ph25_mRp150_23e: 3.96% passed the filter
Topo_2A_qqa_Ph25_mRp300_23e:

In [20]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def Qmake_encoder_set_weights(input_dim,h_dim_1,h_dim_2,latent_dim):
    l2_factor = 1e-3
    inputs = keras.Input(shape=(input_dim), name = "inputs")
    x = QDenseBatchnorm(h_dim_1,
             kernel_quantizer=quantized_bits(10,5,0,alpha=1),
             bias_quantizer=quantized_bits(10,5,0,alpha=1),
             kernel_initializer=keras.initializers.HeNormal(seed=None),
             bias_initializer=keras.initializers.Zeros(),
            name = "dense1")(inputs)
    x = QActivation(quantized_relu(15, negative_slope=1/1024), name = 'relu1')(x)
    x = QDenseBatchnorm(h_dim_2,
             kernel_quantizer=quantized_bits(10,5,0,alpha=1),
             bias_quantizer=quantized_bits(10,5,0,alpha=1),
             kernel_initializer=keras.initializers.HeNormal(seed=None),
             bias_initializer=keras.initializers.Zeros(),
            name = "dense2")(x)
    x = QActivation(quantized_relu(15, negative_slope=1/1024), name = 'relu2')(x)
    z_mean=QDense(latent_dim, name='z_mean',
                  kernel_quantizer=quantized_bits(10,5,0,alpha=1),
                  bias_quantizer=quantized_bits(10,5,0,alpha=1),
                  kernel_initializer=keras.initializers.HeNormal(seed=None),
                  bias_initializer=keras.initializers.Zeros())(x)
    z_logvar=Dense(latent_dim, name='z_log_var',
                          kernel_initializer=keras.initializers.Zeros(),
                          bias_initializer=keras.initializers.Zeros())(x)
    z=Sampling()([z_mean,z_logvar])
    encoder = keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder

def Qmake_decoder_set_weights(input_dim,h_dim_1,h_dim_2,latent_dim):
    l2_factor = 1e-3
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2,
                   activation='relu',
                   kernel_initializer=keras.initializers.HeNormal(seed=None),
                   bias_initializer=keras.initializers.Zeros())(inputs)
    x = BatchNormalization(name="BN_decode1")(x)
    x=layers.Dense(h_dim_1,
                   activation='relu',
                   kernel_initializer=keras.initializers.HeNormal(seed=None),
                   bias_initializer=keras.initializers.Zeros())(x)
    x = BatchNormalization(name="BN_decode2")(x)
    z=layers.Dense(input_dim,
                   kernel_initializer=keras.initializers.HeNormal(seed=None),
                   bias_initializer=keras.initializers.Zeros())(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

def Qmake_discriminator(input_dim, h_dim_1, h_dim_2):
    l2_factor = 1e-3
    inputs = keras.Input(shape=(input_dim))
    x = Dense(h_dim_1,
              activation='relu',
              kernel_initializer=keras.initializers.HeNormal(seed=None),
              bias_initializer=keras.initializers.Zeros())(inputs)
    x = Dense(h_dim_2,
              activation='relu',
              kernel_initializer=keras.initializers.HeNormal(seed=None),
              bias_initializer=keras.initializers.Zeros())(x)
    x = Dense(1,
              activation='sigmoid',  # Output probability
              kernel_initializer=keras.initializers.HeNormal(seed=None),
              bias_initializer=keras.initializers.Zeros())(x)
    discriminator = keras.Model(inputs, x, name='discriminator')
    return discriminator

def custom_mse_loss_with_multi_index_scaling(masked_data, masked_reconstruction):
    jet_scale = 1
    tau_scale = 1
    muon_sacle = 1
    met_scale = 1

    # Define the indices and their corresponding scale factors
    scale_dict = {
        0: jet_scale,
        3: jet_scale,
        6: jet_scale,
        9: jet_scale,
        12: jet_scale,
        15: jet_scale,
        18: tau_scale,
        21: tau_scale,
        24: tau_scale,
        27: tau_scale,
        30: muon_sacle,
        33: muon_sacle,
        36: muon_sacle,
        39: muon_sacle,
        42: met_scale
    }
    
    # Create the scaling tensor
    scale_tensor = tf.ones_like(masked_data)
    
    for index, factor in scale_dict.items():
        index_mask = tf.one_hot(index, depth=tf.shape(masked_data)[-1])
        scale_tensor += index_mask * (factor - 1)
    
    # Apply scaling
    scaled_data = masked_data * scale_tensor
    scaled_reconstruction = masked_reconstruction * scale_tensor
    
#     # Hardcoded lists for eta and phi indices
#     eta_indices = [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40]
#     phi_indices = [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 43]

#     batch_size = tf.shape(scaled_reconstruction)[0]
    
#     # Apply constraints to eta
#     for i in eta_indices:
#         indices = tf.stack([tf.range(batch_size), tf.fill([batch_size], i)], axis=1)
#         updates = 3 * tf.tanh(scaled_reconstruction[:, i] / 3)
#         scaled_reconstruction = tf.tensor_scatter_nd_update(scaled_reconstruction, indices, updates)
    
#     # Apply constraints to phi
#     for i in phi_indices:
#         indices = tf.stack([tf.range(batch_size), tf.fill([batch_size], i)], axis=1)
#         updates = 3.14159265258979*(10/8) * tf.tanh(scaled_reconstruction[:, i] / (3.14159265258979*(10/8)))
#         scaled_reconstruction = tf.tensor_scatter_nd_update(scaled_reconstruction, indices, updates)
    # Calculate MSE using keras.losses.mse
    mse = keras.losses.mse(scaled_data, scaled_reconstruction)
    
    # Take the mean across all dimensions
    return tf.reduce_mean(mse)

class VAE_Model(keras.Model):
    def __init__(self, encoder, decoder, discriminator, steps_per_epoch=20, cycle_length=20, min_beta=0, max_beta=1, min_gamma=0, max_gamma=1, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder        
        self.discriminator = discriminator
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        
        self.discriminator_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
        self.gamma_tracker = keras.metrics.Mean(name="gamma")
        
        self.beta_tracker = keras.metrics.Mean(name="beta")
        self.steps_per_epoch = steps_per_epoch
        self.cycle_length = tf.cast(cycle_length, tf.float32)
        self.min_beta = tf.cast(min_beta, tf.float32)
        self.max_beta = tf.cast(max_beta, tf.float32)
        self.beta = tf.Variable(min_beta, dtype=tf.float32)
        self.min_gamma = tf.cast(min_gamma, tf.float32)
        self.max_gamma = tf.cast(max_gamma, tf.float32)
        self.gamma = tf.Variable(min_gamma, dtype=tf.float32)

    def compile(self, optimizer, **kwargs):
        super(VAE_Model, self).compile(**kwargs)
        # Set the optimizer for the entire model (encoder + decoder + discriminator)
        self.optimizer = optimizer

        # Collect trainable variables from encoder, decoder, and discriminator
        trainable_variables = (
            self.encoder.trainable_weights + 
            self.decoder.trainable_weights + 
            self.discriminator.trainable_weights
        )
        # Build the optimizer with the full variable list
        self.optimizer.build(trainable_variables)

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.discriminator_loss_tracker,
            self.beta_tracker,
        ]
    def cyclical_annealing_beta(self, epoch):
        cycle = tf.floor(1.0 + epoch / self.cycle_length)
        x = tf.abs(epoch / self.cycle_length - cycle + 1)
        # For first half (x < 0.5), scale 2x from 0 to 1
        # For second half (x >= 0.5), stay at 1
        scaled_x = tf.where(x < 0.5, 2.0 * x, 1.0)
        return self.min_beta + (self.max_beta - self.min_beta) * scaled_x
    def get_gamma_schedule(self, epoch):
        # Convert to float32 for TF operations
        epoch = tf.cast(epoch, tf.float32)
        
        # Calculate annealing progress
        anneal_progress = (epoch - 50.0) / 50.0
        gamma_anneal = self.min_gamma + (self.max_gamma - self.min_gamma) * anneal_progress
        
        # Implement the conditions using tf.where
        gamma = tf.where(epoch < 50.0, 
                        0.0,  # if epoch < 50
                        tf.where(epoch >= 100.0,
                                self.max_gamma,  # if epoch >= 100
                                gamma_anneal))   # if 50 <= epoch < 100
        
        return gamma
    
    def train_step(self, data):
        # Get the current epoch number
        epoch = tf.cast(self.optimizer.iterations / self.steps_per_epoch, tf.float32)
        
        # Update beta
        self.beta.assign(self.cyclical_annealing_beta(epoch))
        self.gamma.assign(self.get_gamma_schedule(epoch))

        # ---------------------------
        # Train the Discriminator
        # ---------------------------
        with tf.GradientTape() as tape_disc:
            # Generate reconstructed data
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            
            # Get discriminator predictions
            real_output = self.discriminator(data)
            fake_output = self.discriminator(reconstruction * mask)
            
            # Labels for real and fake data
            real_labels = tf.ones_like(real_output)
            fake_labels = tf.zeros_like(fake_output)
            
            # Discriminator loss
            d_loss_real = keras.losses.binary_crossentropy(real_labels, real_output)
            d_loss_fake = keras.losses.binary_crossentropy(fake_labels, fake_output)
            d_loss = d_loss_real + d_loss_fake
            d_loss = tf.reduce_mean(d_loss)
        
        grads_disc = tape_disc.gradient(d_loss, self.discriminator.trainable_weights)
        self.optimizer.apply_gradients(zip(grads_disc, self.discriminator.trainable_weights))

        # ---------------------------
        # Train the VAE (Generator)
        # ---------------------------
        with tf.GradientTape() as tape:
            data_noisy = data + tf.random.normal(tf.shape(data), 
                                               mean=0, 
                                               stddev=0)
            # Keep the mask logic
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            data_noisy = data_noisy * mask  # Apply mask to noisy data
            
            z_mean, z_log_var, z = self.encoder(data_noisy)
            reconstruction = self.decoder(z)
            reconstruction_loss = custom_mse_loss_with_multi_index_scaling(mask*data, mask*reconstruction)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)

            # Generator (VAE) wants to fool the discriminator
            fake_output = self.discriminator(mask*reconstruction)
            valid_labels = tf.ones_like(fake_output)  # Try to make discriminator think reconstructions are real
            g_loss_adv = keras.losses.binary_crossentropy(valid_labels, fake_output)
            g_loss_adv = tf.reduce_mean(g_loss_adv)

            total_loss = reconstruction_loss + kl_loss * self.beta + g_loss_adv * self.gamma
            
        grads_vae = tape.gradient(total_loss, self.encoder.trainable_weights + self.decoder.trainable_weights)
        self.optimizer.apply_gradients(zip(grads_vae, self.encoder.trainable_weights + self.decoder.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.discriminator_loss_tracker.update_state(g_loss_adv)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "disc_loss": self.discriminator_loss_tracker.result(),
            "beta": self.beta,
            "raw_loss": self.reconstruction_loss_tracker.result() + self.kl_loss_tracker.result(),
            "w_kl_loss": self.kl_loss_tracker.result() * self.beta,
            "w_disc_loss": self.discriminator_loss_tracker.result() * self.gamma,
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = custom_mse_loss_with_multi_index_scaling(mask*data, mask*reconstruction)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(kl_loss)
        # Discriminator loss (only for monitoring)
        # pass both data and reconstruction through D to get generator adversarial loss
        real_output = self.discriminator(data)
        fake_output = self.discriminator(mask*reconstruction)
        real_labels = tf.ones_like(real_output)
        fake_labels = tf.zeros_like(fake_output)
        d_loss_real = keras.losses.binary_crossentropy(real_labels, real_output)
        d_loss_fake = keras.losses.binary_crossentropy(fake_labels, fake_output)
        d_loss = d_loss_real + d_loss_fake
        d_loss = tf.reduce_mean(d_loss)
        
        # Generator adversarial loss
        valid_labels = tf.ones_like(fake_output)
        g_loss_adv = keras.losses.binary_crossentropy(valid_labels, fake_output)
        g_loss_adv = tf.reduce_mean(g_loss_adv)
        total_loss = reconstruction_loss + kl_loss * self.beta + g_loss_adv * self.gamma
        
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
            "raw_loss": reconstruction_loss + kl_loss,
            "disc_loss": g_loss_adv,
            "w_kl_loss": kl_loss * self.beta,
            "w_disc_loss": g_loss_adv * self.gamma
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

In [21]:
def total_loss(signal, predict, z_mean, z_log_var, beta):
    re_loss = np.mean((signal - predict) ** 2)
    kl_loss = np.sum(-0.5 * (1 + z_log_var - (z_mean) ** 2 - np.exp(z_log_var)))
    tot_loss = re_loss + beta * kl_loss
    return tot_loss

def AD_score(s, p, z_m, z_log, beta):
    mask = s != 0

    _s = s * mask
    _p = p * mask

    p = mask * p
    return total_loss(_s, _p, z_m, z_log, beta)


def AD_score_MSE(s, p):
    mask = s != 0
    s1 = s * mask
    p1 = p * mask
    re_loss = np.mean((s1 - p1) ** 2)
    return re_loss


def AD_score_KL(z_mean, z_log_var):
    kl_loss = np.mean(-0.5 * (1 + z_log_var - (z_mean) ** 2 - np.exp(z_log_var)))
    return kl_loss


def AD_score_CKL(z_mean, z_log_var):
    CKL = np.mean(z_mean**2)
    return CKL


class Model_Evaluator:
    def __init__(
        self,
        model_path,
        backround,
        br_weights,
        signal,
        signal_weights,
        input_dim,
        title="placeholder",
        save=False,
        labels=None,
    ):

        self.input_dim = input_dim
        vae_enc = Qmake_encoder_set_weights(input_dim, 32,16,3)
        vae_dec = Qmake_decoder_set_weights(input_dim, 32,16,3)
        vae_disc = Qmake_discriminator(input_dim,8,2)
        self.model = VAE_Model(vae_enc, vae_dec, vae_disc, steps_per_epoch=20, min_beta=0, max_beta=0.1, min_gamma=0.01, max_gamma=0.1)
        self.model.load_weights(model_path).expect_partial()
        self.encoder = self.model.get_layer("encoder")
        self.signal = signal
        self.backround = backround
        self.br_loss = []
        self.signal_loss = []
        self.backround_outputs = []
        self.signal_outputs = []
        self.title = title
        self.saveplots = save
        self.labels = labels
        self.latent_info = []
        self.br_weights = br_weights
        self.signal_weights = signal_weights

    def calculate_loss(self, l_type):
        self.signal_loss = []
        self.br_loss = []
        br = self.backround

        if l_type == "CKL":
            br_latent = np.array(self.encoder.predict(br))
            self.latent_info += [br_latent[0]]
            l = []
            for i in range(0, br.shape[0]):
                loss = AD_score_CKL(br_latent[0][i], br_latent[1][i])
                l += [loss]
            self.br_loss = l

            for i, batch in enumerate(self.signal):
                sg_latent = np.array(self.encoder.predict(batch))
                self.latent_info += [sg_latent[0]]
                l = []

                for i in range(0, batch.shape[0]):
                    loss = AD_score_CKL(sg_latent[0][i], sg_latent[1][i])
                    l += [loss]

                sg_loss = l

                self.signal_loss += [sg_loss]

        if l_type == "KL":
            br_latent = np.array(self.encoder.predict(br))
            l = []
            for i in range(0, br.shape[0]):
                loss = AD_score_KL(br_latent[0][i], br_latent[1][i])
                l += [loss]
            self.br_loss = l

            for i, batch in enumerate(self.signal):
                sg_latent = np.array(self.encoder.predict(batch))

                l = []

                for i in range(0, batch.shape[0]):
                    loss = AD_score_KL(sg_latent[0][i], sg_latent[1][i])
                    l += [loss]

                sg_loss = l

                self.signal_loss += [sg_loss]

        if l_type == "MSE":
            br_predict = np.array(self.model.predict(br)["reconstruction"])
            l = []
            for i in range(0, br.shape[0]):
                loss = AD_score_MSE(br[i], br_predict[i])
                l += [loss]
            self.br_loss = l

            for i, batch in enumerate(self.signal):
                sg_predict = np.array(self.model.predict(batch)["reconstruction"])
                l = []

                for i in range(0, batch.shape[0]):
                    loss = AD_score_MSE(batch[i], sg_predict[i])
                    l += [loss]

                sg_loss = l

                self.signal_loss += [sg_loss]

        if l_type == "MoLV":  # Mean over Log Variance
            br_latent = np.array(self.encoder.predict(br))
            self.latent_info += [br_latent[0]]
            l = []
            for i in range(0, br.shape[0]):
                # Calculate |mean|/log_var for each dimension
                means = np.abs(br_latent[0][i])
                log_vars = br_latent[1][i]  # already log variance
                scores = means/log_vars
                # Take average across dimensions
                l += [np.mean(scores)]
            self.br_loss = l

            for i, batch in enumerate(self.signal):
                sg_latent = np.array(self.encoder.predict(batch))
                self.latent_info += [sg_latent[0]]
                l = []

                for i in range(0, batch.shape[0]):
                    means = np.abs(sg_latent[0][i])
                    log_vars = sg_latent[1][i]
                    scores = means/log_vars
                    l += [np.mean(scores)]

                sg_loss = l
                self.signal_loss += [sg_loss]

        return [self.br_loss, self.signal_loss]

    def ROC(self):
        target_fpr = 1000/(Filter_eff*31575960)
        tpr_at_target = []
        thresholds_at_target = []
        plt.figure(figsize=(12, 8), dpi = 800)
        
        # Get colormap for 18 channels
        colors = cm.tab20(np.linspace(0, 1, len(self.signal_loss)))
        plt.plot(
            (Filter_eff*31575960)*np.linspace(0, 1, 1000), np.linspace(0, 1, 1000), "--", label="diagonal"
        )
        for j, batch in enumerate(self.signal_loss):
            sig_w = self.signal_weights[j]
            br_w = self.br_weights
            weights = np.concatenate((br_w, sig_w))
            truth = []
            for i in range(len(self.br_loss)):
                truth += [0]
            for i in range(len(batch)):
                truth += [1]
            ROC_data = np.concatenate((self.br_loss, batch))
            fpr, tpr, thresholds = sk.roc_curve(truth, ROC_data, sample_weight=weights)
            # auc=np.trapz(tpr,fpr)
            auc = sk.roc_auc_score(truth, ROC_data)
            plt.plot(fpr*Filter_eff*31575960, tpr, label=self.labels[j] + ": " + str(round(auc, 3)),
                color=colors[j])
            
            idx = np.argmin(np.abs(fpr - target_fpr))
            tpr_at_target.append(tpr[idx])
            thresholds_at_target.append(thresholds[idx])

        plt.xlabel("Trigger frequency [Hz]")
        plt.semilogx()
        plt.ylabel("tpr")
        plt.grid()
        plt.semilogy()
        plt.title("{}_ROC".format(self.title))
        plt.vlines(1e3, 0, 1, colors="r", linestyles="dashed")
        plt.legend(bbox_to_anchor=(1.05, 1), 
              loc='upper left', 
              borderaxespad=0.)
        plt.tight_layout()
        if self.saveplots == True:
            plt.savefig(
                "/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/{}_ROC.png".format(
                    self.title
                ),
                format="png",
                bbox_inches="tight",
                dpi=800
            )
        plt.show()
        
        print(f"\nTPR at FPR = {target_fpr} for each channel:")
        for label, tpr, threshold in zip(self.labels, tpr_at_target, thresholds_at_target):
            print(f"{label}: {100*tpr:.4f}%, Theshold = {threshold:.20f}")
    def ROC_loose(self):
        target_fpr = 1000/(Filter_eff*2352*11245)
        tpr_at_target = []
        thresholds_at_target = []
        plt.figure(figsize=(12, 8), dpi = 800)
        
        # Get colormap for 18 channels
        colors = cm.tab20(np.linspace(0, 1, len(self.signal_loss)))
        plt.plot(
            (Filter_eff*2352*11245)*np.linspace(0, 1, 1000), np.linspace(0, 1, 1000), "--", label="diagonal"
        )
        for j, batch in enumerate(self.signal_loss):
            sig_w = self.signal_weights[j]
            br_w = self.br_weights
            weights = np.concatenate((br_w, sig_w))
            truth = []
            for i in range(len(self.br_loss)):
                truth += [0]
            for i in range(len(batch)):
                truth += [1]
            ROC_data = np.concatenate((self.br_loss, batch))
            fpr, tpr, thresholds = sk.roc_curve(truth, ROC_data, sample_weight=weights)
            # auc=np.trapz(tpr,fpr)
            auc = sk.roc_auc_score(truth, ROC_data)
            plt.plot(fpr*Filter_eff*2352*11245, tpr, label=self.labels[j] + ": " + str(round(auc, 3)),
                color=colors[j])
            
            idx = np.argmin(np.abs(fpr - target_fpr))
            tpr_at_target.append(tpr[idx])
            thresholds_at_target.append(thresholds[idx])

        plt.xlabel("Trigger frequency [Hz]")
        plt.semilogx()
        plt.ylabel("tpr")
        plt.grid()
        plt.semilogy()
        plt.title("{}_ROC".format(self.title))
        plt.vlines(1e3, 0, 1, colors="r", linestyles="dashed")
        plt.legend(bbox_to_anchor=(1.05, 1), 
              loc='upper left', 
              borderaxespad=0.)
        plt.tight_layout()
        if self.saveplots == True:
            plt.savefig(
                "/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/{}_ROC.png".format(
                    self.title
                ),
                format="png",
                bbox_inches="tight",
                dpi=800
            )
        plt.show()
        
        print(f"\nTPR at FPR = {target_fpr} for each channel:")
        for label, tpr, threshold in zip(self.labels, tpr_at_target, thresholds_at_target):
            print(f"{label}: {100*tpr:.4f}%, Theshold = {threshold:.20f}")        
    def ROC2k(self):
        target_fpr =2*1000/(Filter_eff*31575960)
        tpr_at_target = []
        thresholds_at_target = []
        plt.figure(figsize=(12, 8), dpi = 800)
        
        # Get colormap for 18 channels
        colors = cm.tab20(np.linspace(0, 1, len(self.signal_loss)))
        plt.plot(
            (Filter_eff*31575960)*np.linspace(0, 1, 1000), np.linspace(0, 1, 1000), "--", label="diagonal"
        )
        for j, batch in enumerate(self.signal_loss):
            sig_w = self.signal_weights[j]
            br_w = self.br_weights
            weights = np.concatenate((br_w, sig_w))
            truth = []
            for i in range(len(self.br_loss)):
                truth += [0]
            for i in range(len(batch)):
                truth += [1]
            ROC_data = np.concatenate((self.br_loss, batch))
            fpr, tpr, thresholds = sk.roc_curve(truth, ROC_data, sample_weight=weights)
            # auc=np.trapz(tpr,fpr)
            auc = sk.roc_auc_score(truth, ROC_data, sample_weight=weights)
            plt.plot(fpr*Filter_eff*31575960, tpr, label=self.labels[j] + ": " + str(round(auc, 3)),
                color=colors[j])
            
            idx = np.argmin(np.abs(fpr - target_fpr))
            tpr_at_target.append(tpr[idx])
            thresholds_at_target.append(thresholds[idx])

        plt.xlabel("Trigger frequency [Hz]")
        plt.semilogx()
        plt.ylabel("tpr")
        plt.grid()
        plt.semilogy()
        plt.title("{}_ROC".format(self.title))
        plt.vlines(2e3, 0, 1, colors="r", linestyles="dashed")
        plt.legend(bbox_to_anchor=(1.05, 1), 
              loc='upper left', 
              borderaxespad=0.)
        plt.tight_layout()
        if self.saveplots == True:
            plt.savefig(
                "/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/{}_ROC.png".format(
                    self.title
                ),
                format="png",
                bbox_inches="tight",
                dpi=800
            )
        plt.show()
        
        print(f"\nTPR at FPR = {target_fpr} for each channel:")
        for label, tpr, threshold in zip(self.labels, tpr_at_target, thresholds_at_target):
            print(f"{label}: {100*tpr:.4f}%, Theshold = {threshold:.20f}")
    def ROC5k(self):
        target_fpr =5*1000/(Filter_eff*31575960)
        tpr_at_target = []
        thresholds_at_target = []
        plt.figure(figsize=(12, 8), dpi = 800)
        
        # Get colormap for 18 channels
        colors = cm.tab20(np.linspace(0, 1, len(self.signal_loss)))
        plt.plot(
            (Filter_eff*31575960)*np.linspace(0, 1, 1000), np.linspace(0, 1, 1000), "--", label="diagonal"
        )
        for j, batch in enumerate(self.signal_loss):
            sig_w = self.signal_weights[j]
            br_w = self.br_weights
            weights = np.concatenate((br_w, sig_w))
            truth = []
            for i in range(len(self.br_loss)):
                truth += [0]
            for i in range(len(batch)):
                truth += [1]
            ROC_data = np.concatenate((self.br_loss, batch))
            fpr, tpr, thresholds = sk.roc_curve(truth, ROC_data, sample_weight=weights)
            # auc=np.trapz(tpr,fpr)
            auc = sk.roc_auc_score(truth, ROC_data, sample_weight=weights)
            plt.plot(fpr*Filter_eff*31575960, tpr, label=self.labels[j] + ": " + str(round(auc, 3)),
                color=colors[j])
            
            idx = np.argmin(np.abs(fpr - target_fpr))
            tpr_at_target.append(tpr[idx])
            thresholds_at_target.append(thresholds[idx])

        plt.xlabel("Trigger frequency [Hz]")
        plt.semilogx()
        plt.ylabel("tpr")
        plt.grid()
        plt.semilogy()
        plt.title("{}_ROC".format(self.title))
        plt.vlines(5e3, 0, 1, colors="r", linestyles="dashed")
        plt.legend(bbox_to_anchor=(1.05, 1), 
              loc='upper left', 
              borderaxespad=0.)
        plt.tight_layout()
        if self.saveplots == True:
            plt.savefig(
                "/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/{}_ROC.png".format(
                    self.title
                ),
                format="png",
                bbox_inches="tight",
                dpi=800
            )
        plt.show()
        
        print(f"\nTPR at FPR = {target_fpr} for each channel:")
        for label, tpr, threshold in zip(self.labels, tpr_at_target, thresholds_at_target):
            print(f"{label}: {100*tpr:.4f}%, Theshold = {threshold:.20f}")
    def ROC10k(self):
        target_fpr =10*1000/(Filter_eff*31575960)
        tpr_at_target = []
        thresholds_at_target = []
        plt.figure(figsize=(12, 8), dpi = 800)
        
        # Get colormap for 18 channels
        colors = cm.tab20(np.linspace(0, 1, len(self.signal_loss)))
        plt.plot(
            (Filter_eff*31575960)*np.linspace(0, 1, 1000), np.linspace(0, 1, 1000), "--", label="diagonal"
        )
        for j, batch in enumerate(self.signal_loss):
            sig_w = self.signal_weights[j]
            br_w = self.br_weights
            weights = np.concatenate((br_w, sig_w))
            truth = []
            for i in range(len(self.br_loss)):
                truth += [0]
            for i in range(len(batch)):
                truth += [1]
            ROC_data = np.concatenate((self.br_loss, batch))
            fpr, tpr, thresholds = sk.roc_curve(truth, ROC_data, sample_weight=weights)
            # auc=np.trapz(tpr,fpr)
            auc = sk.roc_auc_score(truth, ROC_data)
            plt.plot(fpr*Filter_eff*31575960, tpr, label=self.labels[j] + ": " + str(round(auc, 3)),
                color=colors[j])
            
            idx = np.argmin(np.abs(fpr - target_fpr))
            tpr_at_target.append(tpr[idx])
            thresholds_at_target.append(thresholds[idx])

        plt.xlabel("Trigger frequency [Hz]")
        plt.semilogx()
        plt.ylabel("tpr")
        plt.grid()
        plt.semilogy()
        plt.title("{}_ROC".format(self.title))
        plt.vlines(1e4, 0, 1, colors="r", linestyles="dashed")
        plt.legend(bbox_to_anchor=(1.05, 1), 
              loc='upper left', 
              borderaxespad=0.)
        plt.tight_layout()
        if self.saveplots == True:
            plt.savefig(
                "/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/{}_ROC.png".format(
                    self.title
                ),
                format="png",
                bbox_inches="tight",
                dpi=800
            )
        plt.show()
        
        print(f"\nTPR at FPR = {target_fpr} for each channel:")
        for label, tpr, threshold in zip(self.labels, tpr_at_target, thresholds_at_target):
            print(f"{label}: {100*tpr:.4f}%, Theshold = {threshold:.20f}")
    def GetPerformance(self):
        target_fpr = 1000/(Filter_eff*31575960)
        tpr_at_target = []
        thresholds_at_target = []
        auc_array = []

        print(f"Number of signal losses: {len(self.signal_loss)}")
        print(f"Number of labels: {len(self.labels)}")
        print(f"Number of signal weights: {len(self.signal_weights)}")
        print(f"Length of br_loss: {len(self.br_loss)}")

        plt.figure(figsize=(10, 6), dpi= 800)
        plt.plot(Filter_eff*31575960*np.linspace(0, 1, 1000), np.linspace(0, 1, 1000), "--", label="diagonal")
        colors = cm.tab20(np.linspace(0, 1, len(self.signal_loss)))
        for j, batch in enumerate(self.signal_loss):
            print(f"Processing batch {j}")
            print(f"Batch length: {len(batch)}")
            print(f"Signal weight length: {len(self.signal_weights[j])}")

            sig_w = self.signal_weights[j]
            br_w = self.br_weights
            weights = np.concatenate((br_w, sig_w))
            truth = np.concatenate([np.zeros(len(self.br_loss)), np.ones(len(batch))])
            ROC_data = np.concatenate((self.br_loss, batch))

            print(f"ROC_data shape: {ROC_data.shape}")
            print(f"truth shape: {truth.shape}")
            print(f"weights shape: {weights.shape}")

            try:
                fpr, tpr, thresholds = sk.roc_curve(truth, ROC_data, sample_weight=weights)
                auc = sk.roc_auc_score(truth, ROC_data)

                plt.plot(fpr*Filter_eff*31575960, tpr, label=f"{self.labels[j]}: {auc:.3f}", color=colors[j])

                idx = np.argmin(np.abs(fpr - target_fpr))
                tpr_at_target.append(tpr[idx])
                thresholds_at_target.append(thresholds[idx])
                auc_array.append(auc)
                print(f"Successfully processed batch {j}")
            except Exception as e:
                print(f"Error processing batch {j}: {str(e)}")

        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.xscale('log')
        plt.yscale('log')
        plt.title(f"{self.title} ROC")
        plt.vlines(1e3, 0, 1, colors="r", linestyles="dashed")
        plt.legend(loc="lower right")
        if self.saveplots:
            plt.savefig(
                f"/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/analyses_plots/VAE_Varying_Plots/{self.title}_ROC.png",
                format="png",
                bbox_inches="tight",
            )
        plt.show()

        print(f"\nTPR at FPR = {target_fpr} for each channel:")
        results = list(zip(self.labels, tpr_at_target))
        auc_results = list(zip(self.labels, auc_array))
        for label, tpr in results:
            print(f"{label}: {tpr*100:.6f}%")
        print(f"Threshold = {thresholds_at_target[0]:.30f}")
        print(f"Number of results: {len(results)}")
        return results, thresholds_at_target[0], auc_results


In [22]:
with h5py.File("preprocessed_2A_data_raw_FDL_zuizui.h5", "w") as hf:

    # ─── background ───────────────────────────────────────────
    hf.create_dataset('Topo_2A_L1failed',         data=Topo_2A_L1failed)
    hf.create_dataset('Topo_2A_L1failed_mu',      data=Topo_2A_L1failed_mu)
    hf.create_dataset('Topo_EB_L1failed_weights', data=Topo_EB_L1failed_weights)

    # ─── signal (training) ────────────────────────────────────
    hf.create_dataset('Topo_2A_ChiPlusChiMinus100_99_0p3ns', data=Topo_2A_ChiPlusChiMinus100_99_0p3ns)
    hf.create_dataset('Topo_2A_ChiPlusChiMinus500_40_10ns',  data=Topo_2A_ChiPlusChiMinus500_40_10ns)
    hf.create_dataset('Topo_2A_HAHMggf_23e',     data=Topo_2A_HAHMggf_23e)
    hf.create_dataset('Topo_2A_HNLeemu_23e',     data=Topo_2A_HNLeemu_23e)
    hf.create_dataset('Topo_2A_jz2_23e',         data=Topo_2A_jz2_23e)
    hf.create_dataset('Topo_2A_HH4b_23e',        data=Topo_2A_HH4b_23e)
    hf.create_dataset('Topo_2A_ttbar_1lep_23e',  data=Topo_2A_ttbar_1lep_23e)
    hf.create_dataset('Topo_2A_ttbar_2lep_23e',  data=Topo_2A_ttbar_2lep_23e)
    hf.create_dataset('Topo_2A_ZZ4lep_23e',      data=Topo_2A_ZZ4lep_23e)

    # ─── pure‑rate signal channels ────────────────────────────
    hf.create_dataset('Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure', data=Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure)
    hf.create_dataset('Topo_2A_ChiPlusChiMinus500_40_10ns_pure',  data=Topo_2A_ChiPlusChiMinus500_40_10ns_pure)
    hf.create_dataset('Topo_2A_HAHMggf_23e_pure',                 data=Topo_2A_HAHMggf_23e_pure)
    hf.create_dataset('Topo_2A_HNLeemu_23e_pure',                 data=Topo_2A_HNLeemu_23e_pure)
    hf.create_dataset('Topo_2A_jz2_23e_pure',                     data=Topo_2A_jz2_23e_pure)
    hf.create_dataset('Topo_2A_HH4b_23e_pure',                    data=Topo_2A_HH4b_23e_pure)
    hf.create_dataset('Topo_2A_ttbar_1lep_23e_pure',              data=Topo_2A_ttbar_1lep_23e_pure)
    hf.create_dataset('Topo_2A_ttbar_2lep_23e_pure',              data=Topo_2A_ttbar_2lep_23e_pure)
    hf.create_dataset('Topo_2A_ZZ4lep_23e_pure',                  data=Topo_2A_ZZ4lep_23e_pure)
    hf.create_dataset('Topo_2A_CC_directHlR23_150_23e_pure',      data=Topo_2A_CC_directHlR23_150_23e_pure)
    hf.create_dataset('Topo_2A_HtoSUEP_VBF_23e_pure',             data=Topo_2A_HtoSUEP_VBF_23e_pure)
    hf.create_dataset('Topo_2A_ggF_H125_a16a16_23e_pure',         data=Topo_2A_ggF_H125_a16a16_23e_pure)
    hf.create_dataset('Topo_2A_nnlo30_ggH_2a12_23e_pure',         data=Topo_2A_nnlo30_ggH_2a12_23e_pure)
    hf.create_dataset('Topo_2A_ggF_H125_a55a55_100_23e_pure',     data=Topo_2A_ggF_H125_a55a55_100_23e_pure)
    hf.create_dataset('Topo_2A_NNLO_QED_EE_yytautau_23e_pure',    data=Topo_2A_NNLO_QED_EE_yytautau_23e_pure)
    hf.create_dataset('Topo_2A_ggF_H125_a55a55_10_23e_pure',      data=Topo_2A_ggF_H125_a55a55_10_23e_pure)
    hf.create_dataset('Topo_2A_qqa_Ph25_mRp125_23e_pure',         data=Topo_2A_qqa_Ph25_mRp125_23e_pure)
    hf.create_dataset('Topo_2A_qqa_Ph25_mRp150_23e_pure',         data=Topo_2A_qqa_Ph25_mRp150_23e_pure)
    hf.create_dataset('Topo_2A_qqa_Ph25_mRp300_23e_pure',         data=Topo_2A_qqa_Ph25_mRp300_23e_pure)
    hf.create_dataset('Topo_2A_HAHM_S2Zd4e_23e_pure',             data=Topo_2A_HAHM_S2Zd4e_23e_pure)
    hf.create_dataset('Topo_2A_HAHM_S2Zd4mu_23e_pure',            data=Topo_2A_HAHM_S2Zd4mu_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l0cvv1cv1_23e_pure',        data=Topo_2A_hh_vbf_l0cvv1cv1_23e_pure)
    hf.create_dataset('Topo_2A_RS_G_hh_M600_23e_pure',            data=Topo_2A_RS_G_hh_M600_23e_pure)
    hf.create_dataset('Topo_2A_VBF_H125_a55a55_1_23e_pure',       data=Topo_2A_VBF_H125_a55a55_1_23e_pure)
    hf.create_dataset('Topo_2A_ZllH_H125_ctau001_23e_pure',       data=Topo_2A_ZllH_H125_ctau001_23e_pure)
    hf.create_dataset('Topo_2A_ZllH_H125_ctau10_23e_pure',        data=Topo_2A_ZllH_H125_ctau10_23e_pure)
    hf.create_dataset('Topo_2A_Znunu_BFilter_23e_pure',           data=Topo_2A_Znunu_BFilter_23e_pure)
    hf.create_dataset('Topo_2A_Znunu_CFilterBVeto_23e_pure',      data=Topo_2A_Znunu_CFilterBVeto_23e_pure)
    hf.create_dataset('Topo_2A_Zp_ee_ZeroWidth3000_23e_pure',     data=Topo_2A_Zp_ee_ZeroWidth3000_23e_pure)
    hf.create_dataset('Topo_2A_HtoSUEP_ggH_125_23e_pure',         data=Topo_2A_HtoSUEP_ggH_125_23e_pure)
    hf.create_dataset('Topo_2A_HtoSUEP_ggH_500_23e_pure',         data=Topo_2A_HtoSUEP_ggH_500_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l10cvv1cv1_23e_pure',       data=Topo_2A_hh_vbf_l10cvv1cv1_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l1cvv0cv1_23e_pure',        data=Topo_2A_hh_vbf_l1cvv0cv1_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l1cvv1cv1_23e_pure',        data=Topo_2A_hh_vbf_l1cvv1cv1_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l1cvv2cv1_23e_pure',        data=Topo_2A_hh_vbf_l1cvv2cv1_23e_pure)

    # ─── misc flags ───────────────────────────────────────────
    hf.create_dataset('L1_pass_flag', data=L1_pass_flag)

print("Data saved to preprocessed_2A_data_raw_FDL_zuizui.h5")

Data saved to preprocessed_2A_data_raw_FDL.h5


In [23]:
result = [7, 5, 5, 7, 5, 5, 5, 4, 4, 5, 4, 3, 4, 3, 3, 3, 3, 2, 6, 5, 5, 5, 4, 5, 4, 3, 4, 3, 2, 3, 4, 4, 4, 2, 3, 3, 1, 1, 2, -2, -1, -1, 6, 5]
def apply_power_of_2_scaling(X, result):
    # Apply the scaling using 2 raised to the power of the result
    X_scaled = X / (2.0 ** np.array(result))
    return X_scaled
Topo_2A_L1failed = apply_power_of_2_scaling(Topo_2A_L1failed, result)
# Topo_2A_A14_pure = apply_power_of_2_scaling(Topo_2A_A14_pure, result)
# Topo_2A_HAHMggf_pure = apply_power_of_2_scaling(Topo_2A_HAHMggf_pure, result)
# Topo_2A_qqa_pure = apply_power_of_2_scaling(Topo_2A_qqa_pure, result)
# Topo_2A_Zprime_pure = apply_power_of_2_scaling(Topo_2A_Zprime_pure, result)
# Topo_2A_jz1_pure = apply_power_of_2_scaling(Topo_2A_jz1_pure, result)
# Topo_2A_jz2_pure = apply_power_of_2_scaling(Topo_2A_jz2_pure, result)
# Topo_2A_jz4_pure = apply_power_of_2_scaling(Topo_2A_jz4_pure, result)
# Topo_2A_ZZ4lep_pure = apply_power_of_2_scaling(Topo_2A_ZZ4lep_pure, result)
# Topo_2A_HHbbtt_pure = apply_power_of_2_scaling(Topo_2A_HHbbtt_pure, result)

Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure = apply_power_of_2_scaling(Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure, result)
Topo_2A_ChiPlusChiMinus500_40_10ns_pure = apply_power_of_2_scaling(Topo_2A_ChiPlusChiMinus500_40_10ns_pure, result)
Topo_2A_HAHMggf_23e_pure = apply_power_of_2_scaling(Topo_2A_HAHMggf_23e_pure, result)
Topo_2A_HNLeemu_23e_pure = apply_power_of_2_scaling(Topo_2A_HNLeemu_23e_pure, result)
Topo_2A_jz2_23e_pure = apply_power_of_2_scaling(Topo_2A_jz2_23e_pure, result)
Topo_2A_jz4_23e_pure = apply_power_of_2_scaling(Topo_2A_jz4_23e_pure, result)
Topo_2A_HH4b_23e_pure = apply_power_of_2_scaling(Topo_2A_HH4b_23e_pure, result)
Topo_2A_ttbar_1lep_23e_pure = apply_power_of_2_scaling(Topo_2A_ttbar_1lep_23e_pure, result)
Topo_2A_ttbar_2lep_23e_pure = apply_power_of_2_scaling(Topo_2A_ttbar_2lep_23e_pure, result)
Topo_2A_ZZ4lep_23e_pure = apply_power_of_2_scaling(Topo_2A_ZZ4lep_23e_pure, result)

In [24]:
Topo_2A_test = apply_power_of_2_scaling(Topo_2A_test, result)
# Topo_2A_A14 = apply_power_of_2_scaling(Topo_2A_A14, result)
# Topo_2A_HAHMggf = apply_power_of_2_scaling(Topo_2A_HAHMggf, result)
# Topo_2A_qqa = apply_power_of_2_scaling(Topo_2A_qqa, result)
# Topo_2A_Zprime = apply_power_of_2_scaling(Topo_2A_Zprime, result)
# Topo_2A_jz1 = apply_power_of_2_scaling(Topo_2A_jz1, result)
# Topo_2A_jz2 = apply_power_of_2_scaling(Topo_2A_jz2, result)
# Topo_2A_jz4 = apply_power_of_2_scaling(Topo_2A_jz4, result)
# Topo_2A_ZZ4lep = apply_power_of_2_scaling(Topo_2A_ZZ4lep, result)
# Topo_2A_HHbbtt = apply_power_of_2_scaling(Topo_2A_HHbbtt, result)

In [25]:
Topo_2A_CC_directHlR23_150_23e_pure = apply_power_of_2_scaling(Topo_2A_CC_directHlR23_150_23e_pure, result)
Topo_2A_HtoSUEP_VBF_23e_pure = apply_power_of_2_scaling(Topo_2A_HtoSUEP_VBF_23e_pure, result)
Topo_2A_ggF_H125_a16a16_23e_pure = apply_power_of_2_scaling(Topo_2A_ggF_H125_a16a16_23e_pure, result)
Topo_2A_nnlo30_ggH_2a12_23e_pure = apply_power_of_2_scaling(Topo_2A_nnlo30_ggH_2a12_23e_pure, result)
Topo_2A_ggF_H125_a55a55_100_23e_pure = apply_power_of_2_scaling(Topo_2A_ggF_H125_a55a55_100_23e_pure, result)
Topo_2A_NNLO_QED_EE_yytautau_23e_pure = apply_power_of_2_scaling(Topo_2A_NNLO_QED_EE_yytautau_23e_pure, result)
Topo_2A_ggF_H125_a55a55_10_23e_pure = apply_power_of_2_scaling(Topo_2A_ggF_H125_a55a55_10_23e_pure, result)
Topo_2A_qqa_Ph25_mRp125_23e_pure = apply_power_of_2_scaling(Topo_2A_qqa_Ph25_mRp125_23e_pure, result)
Topo_2A_qqa_Ph25_mRp150_23e_pure = apply_power_of_2_scaling(Topo_2A_qqa_Ph25_mRp150_23e_pure, result)
Topo_2A_qqa_Ph25_mRp300_23e_pure = apply_power_of_2_scaling(Topo_2A_qqa_Ph25_mRp300_23e_pure, result)
Topo_2A_HAHM_S2Zd4e_23e_pure = apply_power_of_2_scaling(Topo_2A_HAHM_S2Zd4e_23e_pure, result)
Topo_2A_HAHM_S2Zd4mu_23e_pure = apply_power_of_2_scaling(Topo_2A_HAHM_S2Zd4mu_23e_pure, result)
Topo_2A_RS_G_hh_M1000_23e_pure = apply_power_of_2_scaling(Topo_2A_RS_G_hh_M1000_23e_pure, result)
Topo_2A_hh_vbf_l0cvv1cv1_23e_pure = apply_power_of_2_scaling(Topo_2A_hh_vbf_l0cvv1cv1_23e_pure, result)
Topo_2A_RS_G_hh_M600_23e_pure = apply_power_of_2_scaling(Topo_2A_RS_G_hh_M600_23e_pure, result)
Topo_2A_VBF_H125_a55a55_1_23e_pure = apply_power_of_2_scaling(Topo_2A_VBF_H125_a55a55_1_23e_pure, result)
Topo_2A_ZllH_H125_ctau001_23e_pure = apply_power_of_2_scaling(Topo_2A_ZllH_H125_ctau001_23e_pure, result)
Topo_2A_ZllH_H125_ctau10_23e_pure = apply_power_of_2_scaling(Topo_2A_ZllH_H125_ctau10_23e_pure, result)
Topo_2A_Znunu_BFilter_23e_pure = apply_power_of_2_scaling(Topo_2A_Znunu_BFilter_23e_pure, result)
Topo_2A_Znunu_CFilterBVeto_23e_pure = apply_power_of_2_scaling(Topo_2A_Znunu_CFilterBVeto_23e_pure, result)
Topo_2A_Zp_ee_ZeroWidth3000_23e_pure = apply_power_of_2_scaling(Topo_2A_Zp_ee_ZeroWidth3000_23e_pure, result)
Topo_2A_HtoSUEP_ggH_125_23e_pure = apply_power_of_2_scaling(Topo_2A_HtoSUEP_ggH_125_23e_pure, result)
Topo_2A_HtoSUEP_ggH_500_23e_pure = apply_power_of_2_scaling(Topo_2A_HtoSUEP_ggH_500_23e_pure, result)
Topo_2A_hh_vbf_l10cvv1cv1_23e_pure = apply_power_of_2_scaling(Topo_2A_hh_vbf_l10cvv1cv1_23e_pure, result)
Topo_2A_hh_vbf_l1cvv0cv1_23e_pure = apply_power_of_2_scaling(Topo_2A_hh_vbf_l1cvv0cv1_23e_pure, result)
Topo_2A_hh_vbf_l1cvv1cv1_23e_pure = apply_power_of_2_scaling(Topo_2A_hh_vbf_l1cvv1cv1_23e_pure, result)
Topo_2A_hh_vbf_l1cvv2cv1_23e_pure = apply_power_of_2_scaling(Topo_2A_hh_vbf_l1cvv2cv1_23e_pure, result)


In [26]:
with h5py.File('preprocessed_2A_data_FDL_zuizui.h5', 'w') as hf:
    
    # Background data
    hf.create_dataset('Topo_2A_L1failed', data=Topo_2A_L1failed)
    # hf.create_dataset('Topo_2A_L1failed_mu', data=Topo_2A_L1failed_mu)
    hf.create_dataset('Topo_EB_L1failed_weights', data=Topo_EB_L1failed_weights)
    hf.create_dataset('Topo_2A_test', data=Topo_2A_test)
    hf.create_dataset('Topo_test_weights', data=Topo_test_weights)
    
    # # Signal data
    # hf.create_dataset('Topo_2A_HHbbtt', data=Topo_2A_HHbbtt)
    # hf.create_dataset('Topo_2A_jz1', data=Topo_2A_jz1)
    # hf.create_dataset('Topo_2A_jz2', data=Topo_2A_jz2)
    # hf.create_dataset('Topo_2A_A14', data=Topo_2A_A14)
    # hf.create_dataset('Topo_2A_HAHMggf', data=Topo_2A_HAHMggf)
    # hf.create_dataset('Topo_2A_qqa', data=Topo_2A_qqa)
    # hf.create_dataset('Topo_2A_Zprime', data=Topo_2A_Zprime)
    # hf.create_dataset('Topo_2A_ZZ4lep', data=Topo_2A_ZZ4lep)
    
    # ─── pure‑rate signal channels ────────────────────────────
    hf.create_dataset('Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure', data=Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure)
    hf.create_dataset('Topo_2A_ChiPlusChiMinus500_40_10ns_pure',  data=Topo_2A_ChiPlusChiMinus500_40_10ns_pure)
    hf.create_dataset('Topo_2A_HAHMggf_23e_pure',                 data=Topo_2A_HAHMggf_23e_pure)
    hf.create_dataset('Topo_2A_HNLeemu_23e_pure',                 data=Topo_2A_HNLeemu_23e_pure)
    hf.create_dataset('Topo_2A_jz2_23e_pure',                     data=Topo_2A_jz2_23e_pure)
    hf.create_dataset('Topo_2A_HH4b_23e_pure',                    data=Topo_2A_HH4b_23e_pure)
    hf.create_dataset('Topo_2A_ttbar_1lep_23e_pure',              data=Topo_2A_ttbar_1lep_23e_pure)
    hf.create_dataset('Topo_2A_ttbar_2lep_23e_pure',              data=Topo_2A_ttbar_2lep_23e_pure)
    hf.create_dataset('Topo_2A_ZZ4lep_23e_pure',                  data=Topo_2A_ZZ4lep_23e_pure)
    hf.create_dataset('Topo_2A_CC_directHlR23_150_23e_pure',      data=Topo_2A_CC_directHlR23_150_23e_pure)
    hf.create_dataset('Topo_2A_HtoSUEP_VBF_23e_pure',             data=Topo_2A_HtoSUEP_VBF_23e_pure)
    hf.create_dataset('Topo_2A_ggF_H125_a16a16_23e_pure',         data=Topo_2A_ggF_H125_a16a16_23e_pure)
    hf.create_dataset('Topo_2A_nnlo30_ggH_2a12_23e_pure',         data=Topo_2A_nnlo30_ggH_2a12_23e_pure)
    hf.create_dataset('Topo_2A_ggF_H125_a55a55_100_23e_pure',     data=Topo_2A_ggF_H125_a55a55_100_23e_pure)
    hf.create_dataset('Topo_2A_NNLO_QED_EE_yytautau_23e_pure',    data=Topo_2A_NNLO_QED_EE_yytautau_23e_pure)
    hf.create_dataset('Topo_2A_ggF_H125_a55a55_10_23e_pure',      data=Topo_2A_ggF_H125_a55a55_10_23e_pure)
    hf.create_dataset('Topo_2A_qqa_Ph25_mRp125_23e_pure',         data=Topo_2A_qqa_Ph25_mRp125_23e_pure)
    hf.create_dataset('Topo_2A_qqa_Ph25_mRp150_23e_pure',         data=Topo_2A_qqa_Ph25_mRp150_23e_pure)
    hf.create_dataset('Topo_2A_qqa_Ph25_mRp300_23e_pure',         data=Topo_2A_qqa_Ph25_mRp300_23e_pure)
    hf.create_dataset('Topo_2A_HAHM_S2Zd4e_23e_pure',             data=Topo_2A_HAHM_S2Zd4e_23e_pure)
    hf.create_dataset('Topo_2A_HAHM_S2Zd4mu_23e_pure',            data=Topo_2A_HAHM_S2Zd4mu_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l0cvv1cv1_23e_pure',        data=Topo_2A_hh_vbf_l0cvv1cv1_23e_pure)
    hf.create_dataset('Topo_2A_RS_G_hh_M600_23e_pure',            data=Topo_2A_RS_G_hh_M600_23e_pure)
    hf.create_dataset('Topo_2A_VBF_H125_a55a55_1_23e_pure',       data=Topo_2A_VBF_H125_a55a55_1_23e_pure)
    hf.create_dataset('Topo_2A_ZllH_H125_ctau001_23e_pure',       data=Topo_2A_ZllH_H125_ctau001_23e_pure)
    hf.create_dataset('Topo_2A_ZllH_H125_ctau10_23e_pure',        data=Topo_2A_ZllH_H125_ctau10_23e_pure)
    hf.create_dataset('Topo_2A_Znunu_BFilter_23e_pure',           data=Topo_2A_Znunu_BFilter_23e_pure)
    hf.create_dataset('Topo_2A_Znunu_CFilterBVeto_23e_pure',      data=Topo_2A_Znunu_CFilterBVeto_23e_pure)
    hf.create_dataset('Topo_2A_Zp_ee_ZeroWidth3000_23e_pure',     data=Topo_2A_Zp_ee_ZeroWidth3000_23e_pure)
    hf.create_dataset('Topo_2A_HtoSUEP_ggH_125_23e_pure',         data=Topo_2A_HtoSUEP_ggH_125_23e_pure)
    hf.create_dataset('Topo_2A_HtoSUEP_ggH_500_23e_pure',         data=Topo_2A_HtoSUEP_ggH_500_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l10cvv1cv1_23e_pure',       data=Topo_2A_hh_vbf_l10cvv1cv1_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l1cvv0cv1_23e_pure',        data=Topo_2A_hh_vbf_l1cvv0cv1_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l1cvv1cv1_23e_pure',        data=Topo_2A_hh_vbf_l1cvv1cv1_23e_pure)
    hf.create_dataset('Topo_2A_hh_vbf_l1cvv2cv1_23e_pure',        data=Topo_2A_hh_vbf_l1cvv2cv1_23e_pure)
 
    # Other necessary data
    hf.create_dataset('L1_pass_flag', data=L1_pass_flag)

print("Data saved to preprocessed_2A_data_FDL_zuizui.h5")

Data saved to preprocessed_2A_data_FDL_zuizui.h5


In [63]:
vae_enc = Qmake_encoder_set_weights(44, 32, 16, 3)
vae_dec = Qmake_decoder_set_weights(44, 32, 16, 3)
vae_disc = Qmake_discriminator(44, 8, 2)
vis_model = VAE_Model(vae_enc, vae_dec, vae_disc, steps_per_epoch=20, min_beta=0, max_beta=0.1, min_gamma=0.01, max_gamma=0.1)
vis_model.load_weights('/eos/user/h/hjia/AnomalyDetection/trained_models/L1_models/Topo_2A/versionFDL-GAN_ALT/best_kl_loss/').expect_partial()
vis_encoder = vis_model.get_layer("encoder")

normal_data = Topo_2A_L1failed[:30]

In [ ]:
import plotly.graph_objects as go
import colorsys

In [ ]:
class LatentEvent:
    def __init__(self, data, event_type, channel, index, is_anomalous):
        self.data = data           # The actual event data
        self.event_type = event_type  # 'background' or 'signal'
        self.channel = channel     # Which signal channel or 'background'
        self.index = index         # Original index in dataset
        self.is_anomalous = is_anomalous  # Boolean

In [ ]:
def find_anomalous_events(signals, signal_names, background, encoder, n_top=20):
    """
    Find the most anomalous events from each signal and background
    
    Parameters:
    -----------
    signals: list of numpy arrays
        List of signal datasets
    signal_names: list of str
        Names corresponding to each signal
    background: numpy array
        Background dataset
    encoder: keras model
        Encoder part of VAE
    n_top: int
        Number of top anomalous events to find
        
    Returns:
    --------
    dict: Dictionary containing top anomalous events for each channel
        Structure:
        {
            'background': {
                'data': np.array of events,
                'indices': np.array of original indices,
                'scores': np.array of anomaly scores
            },
            'signal_name1': {
                'data': np.array of events,
                'indices': np.array of original indices,
                'scores': np.array of anomaly scores
            },
            ...
        }
    """
    results = {}
    
    # Process background
    bg_latent = encoder(background)
    bg_mus = bg_latent[0].numpy()
    bg_scores = np.mean(np.square(bg_mus), axis=1)
    bg_top_indices = np.argsort(bg_scores)[-n_top:][::-1]
    
    results['background'] = {
        'data': background[bg_top_indices],
        'indices': bg_top_indices,
        'scores': bg_scores[bg_top_indices]
    }
    
    # Process each signal
    for sig, sig_name in zip(signals, signal_names):
        sig_latent = encoder(sig)
        sig_mus = sig_latent[0].numpy()
        sig_scores = np.mean(np.square(sig_mus), axis=1)
        sig_top_indices = np.argsort(sig_scores)[-n_top:][::-1]
        
        results[sig_name] = {
            'data': sig[sig_top_indices],
            'indices': sig_top_indices,
            'scores': sig_scores[sig_top_indices]
        }
    
    # Print summary
    print("\nAnomalous Events Summary:")
    print("------------------------")
    print(f"\nBackground top {n_top} scores:")
    for i, (idx, score) in enumerate(zip(results['background']['indices'], 
                                       results['background']['scores'])):
        print(f"Event {i+1}: Index {idx}, Score {score:.3f}")
    
    for sig_name in signal_names:
        print(f"\n{sig_name} top {n_top} scores:")
        for i, (idx, score) in enumerate(zip(results[sig_name]['indices'],
                                           results[sig_name]['scores'])):
            print(f"Event {i+1}: Index {idx}, Score {score:.3f}")
            
    return results

# Usage:


In [ ]:
signals = [Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure,
          Topo_2A_ChiPlusChiMinus500_40_10ns_pure,
          Topo_2A_HAHMggf_23e_pure,
          Topo_2A_HNLeemu_23e_pure,
          Topo_2A_jz2_23e_pure,
          Topo_2A_HH4b_23e_pure,
          Topo_2A_ttbar_1lep_23e_pure,
          Topo_2A_ttbar_2lep_23e_pure,
          Topo_2A_ZZ4lep_23e_pure]

signal_names = [
    'ChiPlusChiMinus100_99_0p3ns',
    'ChiPlusChiMinus500_40_10ns',
    'HAHMggf_23e',
    'HNLeemu_23e',
    'jz2_23e',
    'HH4b_23e',
    'ttbar_1lep_23e',
    'ttbar_2lep_23e',
    'ZZ4lep_23e'
]
anomalous_events = find_anomalous_events(
    signals=signals,
    signal_names=signal_names,
    background=Topo_2A_L1failed,
    encoder=vis_encoder
)

In [ ]:
def prepare_events_for_visualization(normal_data, anomalous_events_dict):
    """
    Combine normal and anomalous events into a labeled dataset
    
    Parameters:
    -----------
    normal_data: np.array
        Array of normal events to visualize
    anomalous_events_dict: dict
        Output from find_anomalous_events function
        
    Returns:
    --------
    list of LatentEvent objects
    """
    events_list = []
    
    # Add normal events
    for i, event in enumerate(normal_data):
        events_list.append(LatentEvent(
            data=event,
            event_type='background',
            channel='background',
            index=i,
            is_anomalous=False
        ))
    
    # Add anomalous events from each channel
    for channel, data in anomalous_events_dict.items():
        # Determine event_type based on channel
        event_type = 'background' if channel == 'background' else 'signal'
        
        for event, idx in zip(data['data'], data['indices']):
            events_list.append(LatentEvent(
                data=event,
                event_type=event_type,
                channel=channel,
                index=idx,
                is_anomalous=True
            ))
    
    return events_list

In [ ]:
def generate_gaussian_ellipsoid(mu, sigma, n_points=30):
    """Generate points for a 3D Gaussian ellipsoid surface"""
    u = np.linspace(0, 2*np.pi, n_points)
    v = np.linspace(0, np.pi, n_points)
    
    u, v = np.meshgrid(u, v)
    x = np.cos(u) * np.sin(v)
    y = np.sin(u) * np.sin(v)
    z = np.cos(v)
    
    x = mu[0] + sigma[0] * x
    y = mu[1] + sigma[1] * y
    z = mu[2] + sigma[2] * z
    
    return x, y, z

def generate_colors(n):
    """Generate n distinct colors"""
    colors = []
    for i in range(n):
        hue = i/n
        saturation = 0.7
        value = 0.9
        rgb = colorsys.hsv_to_rgb(hue, saturation, value)
        colors.append(f'rgb({int(rgb[0]*255)},{int(rgb[1]*255)},{int(rgb[2]*255)})')
    return colors

def generate_sphere_surface(radius, n_points=50):
    """Generate a sphere surface for the threshold boundary"""
    phi = np.linspace(0, 2*np.pi, n_points)
    theta = np.linspace(0, np.pi, n_points)
    phi, theta = np.meshgrid(phi, theta)
    
    x = radius * np.cos(phi) * np.sin(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(theta)
    
    return x, y, z
def plot_latent_space_comparison(encoder, labeled_events, threshold=1.0, min_sigma=1e-6):
    """
    Create interactive 3D plot showing labeled events in latent space
    with distinct colors for each signal type
    """
    # Process all events through encoder
    all_data = np.stack([event.data for event in labeled_events])
    latent_vars = encoder(all_data)
    mus = latent_vars[0].numpy()
    sigmas = np.exp(0.5 * latent_vars[1].numpy())
    
    # Calculate plot limits
    max_extent = np.max(np.abs(mus) + 3*sigmas)
    limit = max(max_extent * 1.2, 3)
    
    # Create figure
    fig = go.Figure()
    
    # Add threshold boundary sphere
    boundary_radius = np.sqrt(3 * threshold)
    x, y, z = generate_sphere_surface(boundary_radius)
    fig.add_trace(go.Surface(
        x=x, y=y, z=z,
        colorscale=[[0, 'rgb(200,200,200)'], [1, 'rgb(200,200,200)']],
        showscale=False,
        opacity=0.1,
        name='Threshold Boundary'
    ))
    
    # Add prior distribution
    prior_mu = np.array([0, 0, 0])
    prior_sigma = np.array([1, 1, 1])
    x, y, z = generate_gaussian_ellipsoid(prior_mu, prior_sigma)
    fig.add_trace(go.Surface(
        x=x, y=y, z=z,
        colorscale=[[0, 'rgb(200,200,200)'], [1, 'rgb(200,200,200)']],
        showscale=False,
        opacity=0.2,
        name='Prior Distribution'
    ))
    
    # Create color scheme for different channels
    unique_channels = list(set(event.channel for event in labeled_events))
    channel_colors = {
        'background': 'rgb(30,144,255)',  # Blue for background
    }
    
    # Generate distinct colors for signal channels
    color_list = [
        'rgb(255,0,0)',      # Red
        'rgb(0,255,0)',      # Green
        'rgb(148,0,211)',    # Purple
        'rgb(255,165,0)',    # Orange
        'rgb(0,255,255)',    # Cyan
        'rgb(255,192,203)',  # Pink
        'rgb(128,0,0)',      # Maroon
        'rgb(0,128,0)',      # Dark Green
        'rgb(128,0,128)',    # Purple
    ]
    
    signal_channels = [c for c in unique_channels if c != 'background']
    for i, channel in enumerate(signal_channels):
        channel_colors[channel] = color_list[i % len(color_list)]
    
    # Plot each event
    for i, event in enumerate(labeled_events):
        base_color = channel_colors[event.channel]
        
        # For anomalous events, use a darker version of the same color
        if event.is_anomalous:
            # Parse RGB values
            rgb = base_color.replace('rgb(', '').replace(')', '').split(',')
            r, g, b = map(int, rgb)
            # Darken the color
            r = int(r * 0.7)
            g = int(g * 0.7)
            b = int(b * 0.7)
            color = f'rgb({r},{g},{b})'
        else:
            color = base_color
            
        # Generate and plot ellipsoid
        x, y, z = generate_gaussian_ellipsoid(mus[i], sigmas[i])
        
        name = f"{event.channel} {'(Anomalous)' if event.is_anomalous else '(Normal)'} - Index {event.index}"
        
        fig.add_trace(go.Surface(
            x=x, y=y, z=z,
            colorscale=[[0, color], [1, color]],
            showscale=False,
            opacity=0.3,
            name=name
        ))
        
        # Add center point
        fig.add_trace(go.Scatter3d(
            x=[mus[i,0]], y=[mus[i,1]], z=[mus[i,2]],
            mode='markers',
            marker=dict(size=2, color=color),
            name=name
        ))
    
    # Add coordinate axes
    axis_points = [-limit, limit]
    zeros = [0, 0]
    for points, zeros in zip([axis_points]*3, [[0,0], [0,0], [0,0]]):
        fig.add_trace(go.Scatter3d(
            x=points, y=zeros, z=zeros,
            mode='lines', line=dict(color='black', width=1, dash='dash'),
            showlegend=False
        ))
        fig.add_trace(go.Scatter3d(
            x=zeros, y=points, z=zeros,
            mode='lines', line=dict(color='black', width=1, dash='dash'),
            showlegend=False
        ))
        fig.add_trace(go.Scatter3d(
            x=zeros, y=zeros, z=points,
            mode='lines', line=dict(color='black', width=1, dash='dash'),
            showlegend=False
        ))
    
    # Update layout
    fig.update_layout(
        title='VAE Latent Space Representation',
        width=1400,
        height=1200,
        scene=dict(
            xaxis_title='z₁',
            yaxis_title='z₂',
            zaxis_title='z₃',
            aspectmode='cube',
            camera=dict(
                up=dict(x=0, y=0, z=1),
                center=dict(x=0, y=0, z=0),
                eye=dict(x=1.2, y=1.2, z=1.2)
            ),
            xaxis=dict(range=[-limit, limit]),
            yaxis=dict(range=[-limit, limit]),
            zaxis=dict(range=[-limit, limit])
        ),
        showlegend=True,
        legend=dict(x=1.1, y=0.5)
    )
    
    # Save the figure
    fig.write_html("latent_space_labeled.html")
    
    return fig

# Usage:
# labeled_events = prepare_events_for_visualization(normal_data, anomalous_events)
# fig = plot_latent_space_comparison(vis_encoder, labeled_events)
# fig.show()

In [ ]:
labeled_events = prepare_events_for_visualization(normal_data, anomalous_events)
fig = plot_latent_space_comparison(vis_encoder, labeled_events)
fig.show()

In [27]:
# Topo 2A Analyses
Losses = ['CKL']
for string in Losses:
    evaluation = Model_Evaluator(
        '/eos/user/h/hjia/AnomalyDetection/trained_models/L1_models/Topo_2A/versionFDL-GAN_ALT/best_kl_loss/',
        Topo_2A_L1failed,
        Topo_EB_L1failed_weights,
        [
            Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure,
            Topo_2A_ChiPlusChiMinus500_40_10ns_pure,
            Topo_2A_HAHMggf_23e_pure,
            Topo_2A_HNLeemu_23e_pure,
            Topo_2A_jz2_23e_pure,
            Topo_2A_HH4b_23e_pure,
            Topo_2A_ttbar_1lep_23e_pure,
            Topo_2A_ttbar_2lep_23e_pure,
            Topo_2A_ZZ4lep_23e_pure,
            Topo_2A_CC_directHlR23_150_23e_pure,
            Topo_2A_HtoSUEP_VBF_23e_pure,
            Topo_2A_ggF_H125_a16a16_23e_pure,
            Topo_2A_nnlo30_ggH_2a12_23e_pure,
            Topo_2A_ggF_H125_a55a55_100_23e_pure,
            Topo_2A_NNLO_QED_EE_yytautau_23e_pure,
            Topo_2A_ggF_H125_a55a55_10_23e_pure,
            Topo_2A_qqa_Ph25_mRp125_23e_pure,
            Topo_2A_qqa_Ph25_mRp150_23e_pure,
            Topo_2A_qqa_Ph25_mRp300_23e_pure,
            Topo_2A_HAHM_S2Zd4e_23e_pure,
            Topo_2A_HAHM_S2Zd4mu_23e_pure,
            Topo_2A_hh_vbf_l0cvv1cv1_23e_pure,
            Topo_2A_RS_G_hh_M600_23e_pure,
            Topo_2A_VBF_H125_a55a55_1_23e_pure,
            Topo_2A_ZllH_H125_ctau001_23e_pure,
            Topo_2A_ZllH_H125_ctau10_23e_pure,
            Topo_2A_Znunu_BFilter_23e_pure,
            Topo_2A_Znunu_CFilterBVeto_23e_pure,
            Topo_2A_Zp_ee_ZeroWidth3000_23e_pure,
            Topo_2A_HtoSUEP_ggH_125_23e_pure,
            Topo_2A_HtoSUEP_ggH_500_23e_pure,
            Topo_2A_hh_vbf_l10cvv1cv1_23e_pure,
            Topo_2A_hh_vbf_l1cvv0cv1_23e_pure,
            Topo_2A_hh_vbf_l1cvv1cv1_23e_pure,
            Topo_2A_hh_vbf_l1cvv2cv1_23e_pure
        ],
        [np.ones(len(x)) for x in [
            Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure,
            Topo_2A_ChiPlusChiMinus500_40_10ns_pure,
            Topo_2A_HAHMggf_23e_pure,
            Topo_2A_HNLeemu_23e_pure,
            Topo_2A_jz2_23e_pure,
            Topo_2A_HH4b_23e_pure,
            Topo_2A_ttbar_1lep_23e_pure,
            Topo_2A_ttbar_2lep_23e_pure,
            Topo_2A_ZZ4lep_23e_pure,
            Topo_2A_CC_directHlR23_150_23e_pure,
            Topo_2A_HtoSUEP_VBF_23e_pure,
            Topo_2A_ggF_H125_a16a16_23e_pure,
            Topo_2A_nnlo30_ggH_2a12_23e_pure,
            Topo_2A_ggF_H125_a55a55_100_23e_pure,
            Topo_2A_NNLO_QED_EE_yytautau_23e_pure,
            Topo_2A_ggF_H125_a55a55_10_23e_pure,
            Topo_2A_qqa_Ph25_mRp125_23e_pure,
            Topo_2A_qqa_Ph25_mRp150_23e_pure,
            Topo_2A_qqa_Ph25_mRp300_23e_pure,
            Topo_2A_HAHM_S2Zd4e_23e_pure,
            Topo_2A_HAHM_S2Zd4mu_23e_pure,
            Topo_2A_hh_vbf_l0cvv1cv1_23e_pure,
            Topo_2A_RS_G_hh_M600_23e_pure,
            Topo_2A_VBF_H125_a55a55_1_23e_pure,
            Topo_2A_ZllH_H125_ctau001_23e_pure,
            Topo_2A_ZllH_H125_ctau10_23e_pure,
            Topo_2A_Znunu_BFilter_23e_pure,
            Topo_2A_Znunu_CFilterBVeto_23e_pure,
            Topo_2A_Zp_ee_ZeroWidth3000_23e_pure,
            Topo_2A_HtoSUEP_ggH_125_23e_pure,
            Topo_2A_HtoSUEP_ggH_500_23e_pure,
            Topo_2A_hh_vbf_l10cvv1cv1_23e_pure,
            Topo_2A_hh_vbf_l1cvv0cv1_23e_pure,
            Topo_2A_hh_vbf_l1cvv1cv1_23e_pure,
            Topo_2A_hh_vbf_l1cvv2cv1_23e_pure
        ]],
        input_dim=44,
        title=f'Topo 2A L1AD',
        save=True,
        labels=[
            'ChiPlusChiMinus100_99_0p3ns_23e',
            'ChiPlusChiMinus500_40_10ns_23e',
            'HAHMggf_23e',
            'HNLeemu_23e',
            'jz2_23e',
            'HH4b_23e',
            'ttbar_1lep_23e',
            'ttbar_2lep_23e',
            'ZZ4lep_23e',
            'CC_directHlR23_150_23e',
            'HtoSUEP_VBF_23e',
            'ggF_H125_a16a16_23e',
            'nnlo30_ggH_2a12_23e',
            'ggF_H125_a55a55_100_23e',
            'NNLO_QED_EE_yytautau_23e',
            'ggF_H125_a55a55_10_23e',
            'qqa_Ph25_mRp125_23e',
            'qqa_Ph25_mRp150_23e',
            'qqa_Ph25_mRp300_23e',
            'HAHM_S2Zd4e_23e',
            'HAHM_S2Zd4mu_23e',
            'hh_vbf_l0cvv1cv1_23e',
            'RS_G_hh_M600_23e',
            'VBF_H125_a55a55_1_23e',
            'ZllH_H125_ctau001_23e',
            'ZllH_H125_ctau10_23e',
            'Znunu_BFilter_23e',
            'Znunu_CFilterBVeto_23e',
            'Zp_ee_ZeroWidth3000_23e',
            'HtoSUEP_ggH_125_23e',
            'HtoSUEP_ggH_500_23e',
            'hh_vbf_l10cvv1cv1_23e',
            'hh_vbf_l1cvv0cv1_23e',
            'hh_vbf_l1cvv1cv1_23e',
            'hh_vbf_l1cvv2cv1_23e'
        ]
    )
    A2_loss = evaluation.calculate_loss(string)[1][0]
    evaluation.ROC()

2/2 [==============================] - 0s 2ms/step



TPR at FPR = 3.1775363133638495e-05 for each channel:
ChiPlusChiMinus100_99_0p3ns_23e: 0.5014%, Theshold = 7.11260366439819335938
ChiPlusChiMinus500_40_10ns_23e: 0.2313%, Theshold = 7.11260366439819335938
HAHMggf_23e: 0.4314%, Theshold = 7.11260366439819335938
HNLeemu_23e: 0.4732%, Theshold = 7.11260366439819335938
jz2_23e: 3.7713%, Theshold = 7.11260366439819335938
HH4b_23e: 3.4409%, Theshold = 7.11260366439819335938
ttbar_1lep_23e: 2.2818%, Theshold = 7.11260366439819335938
ttbar_2lep_23e: 1.4493%, Theshold = 7.11260366439819335938
ZZ4lep_23e: 1.1572%, Theshold = 7.11260366439819335938
CC_directHlR23_150_23e: 0.9872%, Theshold = 7.11260366439819335938
HtoSUEP_VBF_23e: 1.8766%, Theshold = 7.11260366439819335938
ggF_H125_a16a16_23e: 1.9563%, Theshold = 7.11260366439819335938
nnlo30_ggH_2a12_23e: 0.1606%, Theshold = 7.11260366439819335938
ggF_H125_a55a55_100_23e: 2.3256%, Theshold = 7.11260366439819335938
NNLO_QED_EE_yytautau_23e: 0.0316%, Theshold = 7.11260366439819335938
ggF_H125_a55

In [ ]:
#Topo 2A Analyses
Losses=['CKL']
for string in Losses:
    evaluation=Model_Evaluator('/eos/user/h/hjia/AnomalyDetection/trained_models/L1_models/Topo_2A/versionFDL-GAN_ALT/best_kl_loss/',
                               Topo_2A_L1failed,
                               Topo_EB_L1failed_weights,
                               [Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure,
                                Topo_2A_ChiPlusChiMinus500_40_10ns_pure,
                                Topo_2A_HAHMggf_23e_pure,
                                Topo_2A_HNLeemu_23e_pure,
                                Topo_2A_jz2_23e_pure,
                                Topo_2A_HH4b_23e_pure,
                                Topo_2A_ttbar_1lep_23e_pure,
                                Topo_2A_ttbar_2lep_23e_pure,
                                Topo_2A_ZZ4lep_23e_pure],
                               [np.ones(len(Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure)),
                                np.ones(len(Topo_2A_ChiPlusChiMinus500_40_10ns_pure)),
                                np.ones(len(Topo_2A_HAHMggf_23e_pure)),
                                np.ones(len(Topo_2A_HNLeemu_23e_pure)),
                                np.ones(len(Topo_2A_jz2_23e_pure)),
                                np.ones(len(Topo_2A_HH4b_23e_pure)),
                                np.ones(len(Topo_2A_ttbar_1lep_23e_pure)),
                                np.ones(len(Topo_2A_ttbar_2lep_23e_pure)),
                                np.ones(len(Topo_2A_ZZ4lep_23e_pure))],
                               input_dim = 44,
                               title=f'Topo 2A L1AD',
                               save = True,
                               labels = ['ChiPlusChiMinus100_99_0p3ns_23e',
                                       'ChiPlusChiMinus500_40_10ns_23e',
                                       'HAHMggf_23e',
                                       'HNLeemu_23e',
                                       'jz2_23e',
                                       'HH4b_23e',
                                       'ttbar_1lep_23e',
                                       'ttbar_2lep_23e',
                                       'ZZ4lep_23e'])
    A2_loss=evaluation.calculate_loss(string)[1][0]
    evaluation.ROC_loose()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from matplotlib.colors import LogNorm
import tensorflow as tf

custom_objects = {
    'QDense': QDense,
    'QActivation': QActivation,
    'QDenseBatchnorm': QDenseBatchnorm,
    'quantized_relu': quantized_relu,
    'quantized_bits': quantized_bits, 
}
loaded_model = tf.keras.models.load_model(
    '/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/2A_AE_model_FDL_GAN_ALT_23e.h5',
    compile=False,# This should allow you to skip compilation for inference
    custom_objects=custom_objects 
)

In [ ]:
signals = [Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure,
          Topo_2A_ChiPlusChiMinus500_40_10ns_pure,
          Topo_2A_HAHMggf_23e_pure,
          Topo_2A_HNLeemu_23e_pure,
          Topo_2A_jz2_23e_pure,
          Topo_2A_HH4b_23e_pure,
          Topo_2A_ttbar_1lep_23e_pure,
          Topo_2A_ttbar_2lep_23e_pure,
          Topo_2A_ZZ4lep_23e_pure]

signal_names = [
    'ChiPlusChiMinus100_99_0p3ns',
    'ChiPlusChiMinus500_40_10ns',
    'HAHMggf_23e',
    'HNLeemu_23e',
    'jz2_23e',
    'HH4b_23e',
    'ttbar_1lep_23e',
    'ttbar_2lep_23e',
    'ZZ4lep_23e'
]

# Process each signal through the model and calculate mean square
signal_results = [np.mean(np.square(loaded_model.predict(sig)), axis=1) for sig in signals]

# Process background
background_result = np.mean(np.square(loaded_model.predict(Topo_2A_L1failed)), axis=1)

In [ ]:
# Create figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

# Determine the range for the bins
all_scores = np.concatenate([background_result] + signal_results)
min_score, max_score = 0, 17.5
# bin_edges = np.logspace(np.log10(min_score), np.log10(max_score), 50)
bin_edges = np.linspace(min_score, max_score, 40)
# Plot background distribution
background_weights = Topo_EB_L1failed_weights / np.sum(Topo_EB_L1failed_weights)  # Normalize weights
ax.hist(background_result, bins=bin_edges, weights=background_weights,
       histtype='step', label='Background (L1failed)',
       color='black', density=True, linewidth=1)

# Plot signal distributions
colors = plt.cm.rainbow(np.linspace(0, 1, len(signal_results)))
for signal, color, name in zip(signal_results, colors, signal_names):
   ax.hist(signal, bins=bin_edges,
           histtype='step', label=name, color=color,
           density=True, linewidth=1)
ax.axvline(x=7.16119766235351562500, color='red', linestyle='--', linewidth=1, label='Threshold')

# Customize plot
# ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Mean Square of Model Output')
ax.set_ylabel('Density')
ax.set_title('Distribution of Model Output Mean Square')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, which='both', linestyle='--', alpha=0.7)

ax.set_ylim(bottom=1e-6, top=2)

# Adjust layout and save
plt.tight_layout()
plt.savefig('model_output_distribution.pdf', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
target_FPR = 1000/(Filter_eff*31575960)  # You'll need to provide Filter_eff

# Sort scores and weights
sorted_indices = np.argsort(background_result)
sorted_scores = background_result[sorted_indices]
sorted_weights = Topo_EB_L1failed_weights[sorted_indices]

# Normalize weights
normalized_weights = sorted_weights / np.sum(sorted_weights)

# Calculate cumulative FPR from highest to lowest score
cumulative_FPR = np.cumsum(normalized_weights[::-1])

# Find where cumulative FPR exceeds target
threshold_index = np.searchsorted(cumulative_FPR, target_FPR)
threshold_value = sorted_scores[::-1][threshold_index]

print(f"Target FPR: {target_FPR}")
print(f"Required threshold value: {threshold_value}")

# Verify the FPR
selected_weights = Topo_EB_L1failed_weights[background_result >= threshold_value]
actual_FPR = np.sum(selected_weights) / np.sum(Topo_EB_L1failed_weights)
print(f"Actual FPR: {actual_FPR}")

In [ ]:
threshold = 7.16119766235351562500

# For each signal, count events above threshold and total length
print("Signal Statistics:")
print("-" * 50)
for signal, name in zip(signal_results, signal_names):
   total_events = len(signal)
   events_above = np.sum(signal >= threshold)
   percentage = (events_above / total_events) * 100
   
   print(f"{name}:")
   print(f"Total events: {total_events}")
   print(f"Events above threshold: {events_above}")
   print(f"Percentage: {percentage:.2f}%")
   print("-" * 50)

# For background
total_bkg = len(background_result)
bkg_above = np.sum(background_result >= threshold)
bkg_percentage = (bkg_above / total_bkg) * 100

print("Background:")
print(f"Total events: {total_bkg}")
print(f"Events above threshold: {bkg_above}")
print(f"Percentage: {bkg_percentage:.2f}%")

In [ ]:
# Find the indices of highest scoring events
n_highest = 1  # number of highest events you want to see
highest_indices = np.argsort(background_result)[-n_highest:]

print("\nHighest scoring background events:")
for idx in reversed(highest_indices):  # reverse to show highest first
   print(f"\nEvent {idx}: Score = {background_result[idx]:.6f}")
   print("Input features:")
   print(Topo_2A_L1failed[idx])

# Get the single highest event
highest_event_idx = highest_indices[-1]
highest_score = background_result[highest_event_idx]
highest_event_features = Topo_2A_L1failed[highest_event_idx]

print(f"\nSingle highest scoring event:")
print(f"Event index: {highest_event_idx}")
print(f"Score: {highest_score:.6f}")
print("Input features:")
for i, value in enumerate(highest_event_features):
   print(f"Feature {i}: {value:.6f}")

In [ ]:
def complete_inversion(X_scaled, result):
    # First invert the power of 2 scaling
    X_temp = X_scaled * (2.0 ** np.array(result))
    X_temp = np.round(X_temp).astype(int)*1.0
    # Then invert the custom scaling
    # For indices divisible by 3 (including 0)
    for i in range(0, len(X_temp), 3):
        X_temp[i] = X_temp[i] / 10
    
    # For indices where mod 3 = 1 (except 43)
    for i in range(1, len(X_temp), 3):
        if i != 43:
            X_temp[i] = X_temp[i] / 40
    
    # For all other indices
    for i in range(len(X_temp)):
        if i % 3 != 0 and (i % 3 != 1 or i == 43):
            X_temp[i] = X_temp[i] * (2 * np.pi / 128)
    
    X = X_temp
    
    return X

In [ ]:
highest_event_L1raw = complete_inversion(highest_event_features, result)

In [ ]:
print(highest_event_L1raw)

In [ ]:
print(Topo_2A_L1failed.shape[0]* 3.138492910903822e-05)
print(Filter_eff)

In [ ]:
results = []
signal_names = ['ChiPlusChiMinus100_99_0p3ns', 'ChiPlusChiMinus500_40_10ns',
                'HAHMggf_23e', 'HNLeemu_23e', 'jz2_23e', 'HH4b_23e', 
                'ttbar_1lep_23e', 'ttbar_2lep_23e', 'ZZ4lep_23e']
channel_results = {}
auc_channel_results = {}
thresholds_model = []

for i in range(30):
    try:
        model_path = f'/eos/user/h/hjia/trained_models/L1_models/Repeat_Testing/2A_FDL_GANALT_{i}/'
        print(f"Evaluating model from path: {model_path}")
        
        evaluation = Model_Evaluator(model_path,
                                   Topo_2A_L1failed,
                                   Topo_EB_L1failed_weights,
                                   [Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure,
                                    Topo_2A_ChiPlusChiMinus500_40_10ns_pure,
                                    Topo_2A_HAHMggf_23e_pure,
                                    Topo_2A_HNLeemu_23e_pure,
                                    Topo_2A_jz2_23e_pure,
                                    Topo_2A_HH4b_23e_pure,
                                    Topo_2A_ttbar_1lep_23e_pure,
                                    Topo_2A_ttbar_2lep_23e_pure,
                                    Topo_2A_ZZ4lep_23e_pure],
                                   [np.ones(len(Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure)),
                                    np.ones(len(Topo_2A_ChiPlusChiMinus500_40_10ns_pure)),
                                    np.ones(len(Topo_2A_HAHMggf_23e_pure)),
                                    np.ones(len(Topo_2A_HNLeemu_23e_pure)),
                                    np.ones(len(Topo_2A_jz2_23e_pure)),
                                    np.ones(len(Topo_2A_HH4b_23e_pure)),
                                    np.ones(len(Topo_2A_ttbar_1lep_23e_pure)),
                                    np.ones(len(Topo_2A_ttbar_2lep_23e_pure)),
                                    np.ones(len(Topo_2A_ZZ4lep_23e_pure))],
                                   input_dim = 44,
                                   title=f'Topo 2A Model {i}',
                                   save = False,
                                   labels = ['ChiPlusChiMinus100_99_0p3ns',
                                           'ChiPlusChiMinus500_40_10ns',
                                           'HAHMggf_23e',
                                           'HNLeemu_23e',
                                           'jz2_23e',
                                           'HH4b_23e',
                                           'ttbar_1lep_23e',
                                           'ttbar_2lep_23e',
                                           'ZZ4lep_23e'])
        
        print("Model_Evaluator instance created successfully")
        
        print("Calculating losses...")
        evaluation.calculate_loss('CKL')
        print("Losses calculated")
        
        print("Getting performance...")
        result, temp_thresh, auc_results = evaluation.GetPerformance()
        print(f"GetPerformance() returned: {result}")
        thresholds_model.append(temp_thresh)
        if not result:
            print(f"Warning: GetPerformance() returned an empty result for model {i}")
        else:
            for channel, tpr in result:
                if channel not in channel_results:
                    channel_results[channel] = []
                channel_results[channel].append(tpr)

        if not auc_results:
            print(f"Warning: GetPerformance() returned empty AUC results for model {i}")
        else:
            for channel, auc in auc_results:
                if channel not in auc_channel_results:
                    auc_channel_results[channel] = []
                auc_channel_results[channel].append(auc)
    
    except Exception as e:
        print(f"An error occurred while evaluating model {i}:")
        print(e)

print("Final channel results:", channel_results)
print("Final auc channel results:", auc_channel_results)

In [ ]:
# results = []
# signal_names = ['HNL', 'HAHMggf', 'qqa', 'Zprime', 'ZZ4lep', 'HHbbtt', 'jz1', 'jz2', 
#                 'ChiPlusChiMinus100_99_0p3ns', 'ChiPlusChiMinus500_40_10ns',
#                 'HAHMggf_23e', 'HNLeemu_23e', 'jz2_23e', 'HH4b_23e', 
#                 'ttbar_1lep_23e', 'ttbar_2lep_23e', 'ZZ4lep_23e']
# channel_results = {}
# auc_channel_results = {}
# thresholds_model = []

# for i in range(50):
#     try:
#         model_path = f'/eos/user/h/hjia/trained_models/L1_models/Repeat_Testing/2A_FDL_GAN_{i}/'
#         print(f"Evaluating model from path: {model_path}")
        
#         evaluation = Model_Evaluator(model_path,
#                                    Topo_2A_L1failed,
#                                    Topo_EB_L1failed_weights,
#                                    [Topo_2A_A14_pure,
#                                     Topo_2A_HAHMggf_pure,
#                                     Topo_2A_qqa_pure,
#                                     Topo_2A_Zprime_pure,
#                                     Topo_2A_ZZ4lep_pure,
#                                     Topo_2A_HHbbtt_pure,
#                                     Topo_2A_jz1_pure,
#                                     Topo_2A_jz2_pure,
#                                     Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure,
#                                     Topo_2A_ChiPlusChiMinus500_40_10ns_pure,
#                                     Topo_2A_HAHMggf_23e_pure,
#                                     Topo_2A_HNLeemu_23e_pure,
#                                     Topo_2A_jz2_23e_pure,
#                                     Topo_2A_HH4b_23e_pure,
#                                     Topo_2A_ttbar_1lep_23e_pure,
#                                     Topo_2A_ttbar_2lep_23e_pure,
#                                     Topo_2A_ZZ4lep_23e_pure],
#                                    [np.ones(len(Topo_2A_A14_pure)),
#                                     np.ones(len(Topo_2A_HAHMggf_pure)),
#                                     np.ones(len(Topo_2A_qqa_pure)),
#                                     np.ones(len(Topo_2A_Zprime_pure)),
#                                     np.ones(len(Topo_2A_ZZ4lep_pure)),
#                                     np.ones(len(Topo_2A_HHbbtt_pure)),
#                                     np.ones(len(Topo_2A_jz1_pure)),
#                                     np.ones(len(Topo_2A_jz2_pure)),
#                                     np.ones(len(Topo_2A_ChiPlusChiMinus100_99_0p3ns_pure)),
#                                     np.ones(len(Topo_2A_ChiPlusChiMinus500_40_10ns_pure)),
#                                     np.ones(len(Topo_2A_HAHMggf_23e_pure)),
#                                     np.ones(len(Topo_2A_HNLeemu_23e_pure)),
#                                     np.ones(len(Topo_2A_jz2_23e_pure)),
#                                     np.ones(len(Topo_2A_HH4b_23e_pure)),
#                                     np.ones(len(Topo_2A_ttbar_1lep_23e_pure)),
#                                     np.ones(len(Topo_2A_ttbar_2lep_23e_pure)),
#                                     np.ones(len(Topo_2A_ZZ4lep_23e_pure))],
#                                    input_dim = 44,
#                                    title=f'Topo 2A Model {i}',
#                                    save = False,
#                                    labels = ['HNL_pure',
#                                            'HAHMggf_pure',
#                                            'qqa_pure',
#                                            'Zprime_pure',
#                                            'ZZ4lep_pure',
#                                            'HHbbtt_pure',
#                                            'jz1',
#                                            'jz2',
#                                            'ChiPlusChiMinus100_99_0p3ns',
#                                            'ChiPlusChiMinus500_40_10ns',
#                                            'HAHMggf_23e',
#                                            'HNLeemu_23e',
#                                            'jz2_23e',
#                                            'HH4b_23e',
#                                            'ttbar_1lep_23e',
#                                            'ttbar_2lep_23e',
#                                            'ZZ4lep_23e'])
        
#         print("Model_Evaluator instance created successfully")
        
#         print("Calculating losses...")
#         evaluation.calculate_loss('CKL')
#         print("Losses calculated")
        
#         print("Getting performance...")
#         result, temp_thresh, auc_results = evaluation.GetPerformance()
#         print(f"GetPerformance() returned: {result}")
#         thresholds_model.append(temp_thresh)
#         if not result:
#             print(f"Warning: GetPerformance() returned an empty result for model {i}")
#         else:
#             for channel, tpr in result:
#                 if channel not in channel_results:
#                     channel_results[channel] = []
#                 channel_results[channel].append(tpr)

#         if not auc_results:
#             print(f"Warning: GetPerformance() returned empty AUC results for model {i}")
#         else:
#             for channel, auc in auc_results:
#                 if channel not in auc_channel_results:
#                     auc_channel_results[channel] = []
#                 auc_channel_results[channel].append(auc)
    
#     except Exception as e:
#         print(f"An error occurred while evaluating model {i}:")
#         print(e)

# print("Final channel results:", channel_results)
# print("Final auc channel results:", auc_channel_results)

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'model_index': range(30),
    'threshold': thresholds_model
})
df.to_csv('standard_threshold_FDL.csv', index=False)

In [ ]:
print(thresholds_model)

In [ ]:
print(len(thresholds_model))

In [ ]:
if channel_results:
    plt.figure(figsize=(12, 6))
    channels = list(channel_results.keys())
    n_channels = len(channels)
    n_models = len(channel_results[channels[0]])
    
    x = np.arange(n_models)
    width = 0.8 / n_channels
    
    for i, channel in enumerate(channels):
        tprs = channel_results[channel]
        plt.bar(x + i * width, tprs, width, label=channel)
    
    plt.xlabel('Models')
    plt.ylabel('True Positive Rate (TPR) at FPR = 1e-5')
    plt.title('TPR Comparison Across Models and Channels')
    plt.xticks(x + width * (n_channels - 1) / 2, [f'{i}' for i in range(n_models)])
    plt.legend()
    plt.tight_layout()
    plt.savefig('tpr_comparison_plot.png', dpi=600, bbox_inches='tight')
    plt.show()
    
    # Print statistics
    print("\nChannel Statistics:")
    for channel in channels:
        tprs = channel_results[channel]
        mean = np.mean(tprs)
        std = np.std(tprs)
        print(f"{channel}: Mean TPR = {mean:.4f}, Std Dev = {std:.4f}")
else:
    print("No results were obtained. Cannot create comparison plot.")

In [ ]:
if channel_results:
    plt.figure(figsize=(14, 8))  # Increased figure size to accommodate the legend
    channels = list(channel_results.keys())
    n_channels = len(channels)
    n_models = len(channel_results[channels[0]])
    
    for i, channel in enumerate(channels):
        tprs = channel_results[channel]
        plt.scatter(range(n_models), tprs, label=channel, s=50)
    
    plt.xlabel('Models')
    plt.ylabel('True Positive Rate (TPR) at FPR = 1e-5')
    plt.semilogy()
    plt.title('TPR Comparison Across Models and Channels')
    plt.xticks(range(n_models), [f'{i}' for i in range(n_models)])
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Place legend outside the plot
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.savefig('tpr_comparison_plot.png', dpi=600, bbox_inches='tight')
    plt.show()
    
    # Print statistics
    print("\nChannel Statistics:")
    for channel in channels:
        tprs = channel_results[channel]
        mean = np.mean(tprs)
        std = np.std(tprs)
        print(f"{channel}: Mean TPR = {mean:.4f}, Std Dev = {std:.4f}")
else:
    print("No results were obtained. Cannot create comparison plot.")

In [ ]:
if auc_channel_results:
    plt.figure(figsize=(14, 8))  # Increased figure size to accommodate the legend
    channels = list(auc_channel_results.keys())
    n_channels = len(channels)
    n_models = len(auc_channel_results[channels[0]])
    
    for i, channel in enumerate(channels):
        aucs = auc_channel_results[channel]
        plt.scatter(range(n_models), aucs, label=channel, s=50)
    
    plt.xlabel('Models')
    plt.ylabel('AUC')
    # plt.semilogy()
    plt.ylim(0.6, 1)
    plt.title('AUC Comparison Across Models and Channels')
    plt.xticks(range(n_models), [f'{i}' for i in range(n_models)])
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Place legend outside the plot
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.savefig('auc_comparison_plot.png', dpi=600, bbox_inches='tight')
    plt.show()
    
    # Print statistics
    print("\nChannel Statistics:")
    for channel in channels:
        aucs = auc_channel_results[channel]
        mean = np.mean(aucs)
        std = np.std(aucs)
        print(f"{channel}: Mean TPR = {mean:.4f}, Std Dev = {std:.4f}")
else:
    print("No results were obtained. Cannot create comparison plot.")

In [ ]:
def plot_distribution_outside(br_loss, br_weights, signal_losses, signal_weights, labels, 
                                    title, saveplots=False, save_path=None):
    fig, ax = plt.subplots(figsize=(12, 8))
    bins = 100
    
    # Determine the range for the bins
    all_scores = np.concatenate([br_loss] + signal_losses)
    min_score, max_score = 0.1, 1e2
    bin_edges = np.linspace(min_score, max_score, bins + 1)

    # Plot signal distributions
    colors = plt.cm.rainbow(np.linspace(0, 1, len(signal_losses)))
    for i, (signal_scores, signal_weights, label) in enumerate(zip(signal_losses, signal_weights, labels)):
        signal_weights = signal_weights / np.sum(signal_weights)  # Normalize weights
        ax.hist(signal_scores, bins=bin_edges, weights=signal_weights, 
                histtype='step', label=label, color=colors[i],
                density=True, linewidth=2)
    
    # Plot background distribution
    br_weights = br_weights / np.sum(br_weights)  # Normalize weights
    ax.hist(br_loss, bins=bin_edges, weights=br_weights, histtype='step', 
            label='Background', color='black', density=True, linewidth=2)
    
    ax.set_xlabel('Anomaly Score')
    ax.set_ylabel('Weighted Normalized Density')
    ax.set_title(f'{title} Weighted Normalized Anomaly Score Distribution')
    ax.legend()
    ax.grid(True, which='both', linestyle='--', alpha=0.7)
    ax.set_ylim(bottom=1e-5, top=10)
    ax.set_xscale('log')
    ax.set_yscale('log')
    
    if saveplots and save_path:
        plt.savefig(save_path, format="png", bbox_inches="tight")
    plt.show()


In [ ]:
# ori_vae_enc = Qmake_encoder_set_weights(Topo_2A_test_br.shape[1],32,16,3)
# ori_vae_dec = Qmake_decoder_set_weights(Topo_2A_test_br.shape[1],32,16,3)
ori_vae_enc = Qmake_encoder_set_weights(44,32,16,3)
ori_vae_dec = Qmake_decoder_set_weights(44,32,16,3)
ori_disc = Qmake_discriminator(44,8,2)
orginal_model = VAE_Model(ori_vae_enc, ori_vae_dec, ori_disc)

orginal_model.load_weights("/eos/user/h/hjia/AnomalyDetection/trained_models/L1_models/Topo_2A/versionFDL-GAN_ALT/best_kl_loss/")
def make_simplified_encoder(input_dim, h_dim_1, h_dim_2, latent_dim):
    inputs = keras.Input(shape=(input_dim,), name='inputs')
    x = QDenseBatchnorm(h_dim_1,
            kernel_quantizer=quantized_bits(10,5,0,alpha=1),
            bias_quantizer=quantized_bits(10,5,0,alpha=1),
            name = "dense1")(inputs)
    x = QActivation(quantized_relu(15, negative_slope=1/1024), name = 'relu1')(x)
    x = QDenseBatchnorm(h_dim_2,
            kernel_quantizer=quantized_bits(10,5,0,alpha=1),
            bias_quantizer=quantized_bits(10,5,0,alpha=1),
            name = "dense2")(x)
    x = QActivation(quantized_relu(15, negative_slope=1/1024), name = 'relu2')(x)
    z_mean = QDense(latent_dim, name='z_mean',
                  kernel_quantizer=quantized_bits(10,5,0,alpha=1),
                  bias_quantizer=quantized_bits(10,5,0,alpha=1))(x)
    new_encoder = keras.Model(inputs, z_mean, name='simplified_encoder')
    return new_encoder

In [ ]:
# Create the new encoder
# new_encoder = make_simplified_encoder(Topo_2A_test_br.shape[1],32,16,3)
new_encoder = make_simplified_encoder(44,32,16,3)
# Transfer weights from the original encoder
for layer in new_encoder.layers:
    original_layer = ori_vae_enc.get_layer(layer.name)
    if original_layer is not None:
        print("set weight for ", layer)
        print(original_layer.get_weights())
        layer.set_weights(original_layer.get_weights())

In [ ]:
tf.keras.models.save_model(
    new_encoder, 
    '/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/2A_AE_model_FDL_GAN_ALT_23e.h5',
    save_format='h5',
    include_optimizer=False  # Since you only need it for inference
)

In [ ]:
tf.keras.models.save_model(
    new_encoder, 
    '2A_AE_model_FDL_GAN_23e',
    save_format='tf',
    include_optimizer=False  # Since you only need it for inference
)

In [ ]:
L1failed_predict = new_encoder.predict(Topo_2A_L1failed)

In [ ]:
print(Topo_2A_L1failed.shape)

In [ ]:
def calculate_pair_mass(events_array, idx_start, pt1_scale=7, eta1_scale=4, phi1_scale=5, 
                       pt2_scale=7, eta2_scale=3, phi2_scale=4):
    # Extract and rescale kinematics
    pt1 = events_array[:, idx_start] * (2.0 ** pt1_scale)/10.0
    eta1 = events_array[:, idx_start + 1] * (2.0 ** eta1_scale)/40.0
    phi1 = events_array[:, idx_start + 2] * (2.0 ** phi1_scale)/phi_res
    pt2 = events_array[:, idx_start + 3] * (2.0 ** pt2_scale)/10.0
    eta2 = events_array[:, idx_start + 4] * (2.0 ** eta2_scale)/40.0
    phi2 = events_array[:, idx_start + 5] * (2.0 ** phi2_scale)/phi_res
    
    # Calculate M² using the formula
    deta = eta1 - eta2  # array subtraction
    dphi = phi1 - phi2  # array subtraction
    
    M_squared = 2 * pt1 * pt2 * (np.cosh(deta) - np.cos(dphi))
    
    return np.sqrt(M_squared)

In [ ]:
# test_data = np.vstack((Topo_2A_L1failed, Topo_2A_HAHMggf_pure, Topo_2A_qqa_pure, Topo_2A_A14_pure, Topo_2A_Zprime_pure))
test_data = Topo_2A_L1failed
# test_data = data
output_data = loaded_model.predict(test_data)
print(output_data.shape)
leadjetPT = test_data[:, 0]
subleadjetPT = test_data[:, 3]
leadtauPT = test_data[:, 18]
leadmuonPT = test_data[:, 30]
METvalue = test_data[:, 42]
columns_of_interest = [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42]
totalPT = np.sum(test_data[:, columns_of_interest], axis=1)
jetpairmass = calculate_pair_mass(test_data, 0, pt1_scale=7, eta1_scale=5, phi1_scale=5, pt2_scale=7, eta2_scale=5, phi2_scale=5)
taupairmass = calculate_pair_mass(test_data, 18, pt1_scale=6, eta1_scale=5, phi1_scale=5, pt2_scale=5, eta2_scale=4, phi2_scale=5)
muonpairmass = calculate_pair_mass(test_data, 30, pt1_scale=4, eta1_scale=4, phi1_scale=4, pt2_scale=2, eta2_scale=3, phi2_scale=3)

In [ ]:
from scipy.stats import binomtest
def calculate_efficiency_with_errors(mu_values, scores, threshold, nbins=20):
    # Create mu bins
    mu_bins = np.linspace(np.min(mu_values), np.max(mu_values), nbins + 1)
    bin_centers = (mu_bins[1:] + mu_bins[:-1]) / 2
    
    efficiencies = []
    errors_up = []
    errors_down = []
    
    # Calculate efficiency for each bin
    for i in range(len(mu_bins)-1):
        mask = (mu_values >= mu_bins[i]) & (mu_values < mu_bins[i+1])
        if np.sum(mask) == 0:  # Skip empty bins
            efficiencies.append(0)
            errors_up.append(0)
            errors_down.append(0)
            continue
            
        total = np.sum(mask)
        passed = np.sum((scores[mask] >= threshold))
        
        # Calculate efficiency and confidence intervals using binomial statistics
        if total > 0:
            result = binomtest(passed, total)
            ci = result.proportion_ci(confidence_level=0.68)  # Using 1-sigma confidence interval
            eff = passed / total
            efficiencies.append(eff)
            errors_up.append(ci.high - eff)
            errors_down.append(eff - ci.low)
        else:
            efficiencies.append(0)
            errors_up.append(0)
            errors_down.append(0)
    
    return bin_centers, np.array(efficiencies), np.array(errors_up), np.array(errors_down)

In [ ]:
# Calculate multiplicity (number of non-zero values) for each row, but only for specified columns
multiplicities = np.count_nonzero(test_data[:, columns_of_interest], axis=1)
square = output_data**2
print(square.shape)
# Calculate the mean of sum of squares of the three outputs
output_mean_sum_squares = np.mean(output_data**2, axis=1)

# Calculate correlation
correlation, _ = pearsonr(multiplicities, output_mean_sum_squares)
# disco = dcor(multiplicities, output_mean_sum_squares)
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(multiplicities, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel('multiplicities')
plt.ylabel('AD score')
plt.title('Density Plot: Multiplicities vs AD score')

# Add correlation information to the plot
plt.text(0.05, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/multi.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(multiplicities, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('multiplicities')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim([1e-6, 1e2])
plt.title('L1AD Efficiency vs multiplicities')
plt.grid(True, alpha=0.3)
plt.legend()

# Set y-axis limits with some padding
# plt.ylim(-0.05, 1.05)

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/multi_eff.png')

In [ ]:
correlation, _ = pearsonr(Topo_2A_L1failed_mu, output_mean_sum_squares)
# disco = dcor(leadjetPT, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(Topo_2A_L1failed_mu, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel('mu')
plt.ylabel('AD score')
plt.title('Density Plot: mu vs AD score')

# Add correlation information to the plot
plt.text(0.05, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/mu2d_nofilt.png')

In [ ]:
correlation, _ = pearsonr(Topo_2A_L1failed_mu, multiplicities)
# disco = dcor(leadjetPT, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(Topo_2A_L1failed_mu, multiplicities, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel('mu')
plt.ylabel('multiplicities')
plt.title('Density Plot: mu vs multiplicities')

# Add correlation information to the plot
plt.text(0.05, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/mu_multi.png')

In [ ]:
def plot_efficiency(mu_values, scores, threshold):
    plt.figure(figsize=(10, 6), dpi=800)
    
    # Calculate efficiencies and errors
    bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(mu_values, scores, threshold)
    
    # Plot efficiency points with error bars
    plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
                 capsize=3, capthick=1, elinewidth=1, markersize=4,
                 label=f'Threshold ≥ {threshold:.5f} (Correspond to 1kHz@L1AD)')
    
    # Add horizontal line at zero
    plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    
    # Customize plot
    plt.xlabel('μ')
    plt.ylabel('Efficiency')
    plt.yscale('log')
    plt.ylim([1e-6, 1])
    plt.title('L1AD Efficiency vs μ')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Set y-axis limits with some padding
    # plt.ylim(-0.05, 1.05)
    
    plt.tight_layout()
    plt.show()
    plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/mu_eff.png')

In [ ]:
plot_efficiency(Topo_2A_L1failed_mu, output_mean_sum_squares, threshold=7.16119766235351562500)

In [ ]:
correlation, _ = pearsonr(leadjetPT*12.8, output_mean_sum_squares)
# disco = dcor(leadjetPT, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(leadjetPT*12.8, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel('Leading Jet pT [GeV]')
plt.ylabel('AD score')
plt.title('Density Plot: Leading Jet pT vs AD score')

# Add correlation information to the plot
plt.text(0.75, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/leadJet.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(leadjetPT*12.8, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('leading jet PT [GeV]')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim(1e-6,10)
plt.title('L1AD Efficiency vs Leading Jet PT')
plt.grid(True, alpha=0.3)
plt.legend()

# Set y-axis limits with some padding
# plt.ylim(-0.05, 1.05)

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/jet_eff.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(leadmuonPT, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('leading muon PT')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim(1e-6,10)
plt.title('L1AD Efficiency vs Leading muon PT')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/muon_eff.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(leadtauPT, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('leading tau PT')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim(1e-6,10)
plt.title('L1AD Efficiency vs Leading tau PT')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/tau_eff.png')

In [ ]:
correlation, _ = pearsonr(leadmuonPT, output_mean_sum_squares)
# disco = dcor(leadmuonPT, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(leadmuonPT, output_mean_sum_squares, bins=20, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel('Leading muon pT')
plt.ylabel('AD score')
plt.title('Density Plot: Leading Muon pT vs AD score')

# Add correlation information to the plot
plt.text(0.75, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/leadMuon.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(METvalue, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('MET')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim(1e-6, 10)
plt.title('L1AD Efficiency vs MET')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/MET_eff.png')

In [ ]:
correlation, _ = pearsonr(leadtauPT, output_mean_sum_squares)
# disco = dcor(leadtauPT, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(leadtauPT, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel('Leading tau pT')
plt.ylabel('AD score')
plt.title('Density Plot: Leading Tau pT vs AD score')

# Add correlation information to the plot
plt.text(0.75, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/leadTau.png')

In [ ]:
correlation, _ = pearsonr(METvalue , output_mean_sum_squares)
# disco = dcor(METvalue, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(METvalue , output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel('MET')
plt.ylabel('AD score')
plt.title('Density Plot: MET vs AD score')

# Add correlation information to the plot
plt.text(0.75, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/MET_corr.png')

In [ ]:
correlation, _ = pearsonr(jetpairmass, output_mean_sum_squares)
# disco = dcor(METvalue, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(jetpairmass, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel(' leading jet pair invariant mass[GeV]')
plt.ylabel('AD score')
plt.title('Density Plot: m_jj vs AD score')

# Add correlation information to the plot
plt.text(0.75, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/jetpair.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(jetpairmass, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('Leading jet pair invariant mass [GeV]')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim(1e-6,10)
plt.title('L1AD Efficiency vs Leading jet pair invariant mass')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/jetpairmass_eff.png')

In [ ]:
correlation, _ = pearsonr(taupairmass, output_mean_sum_squares)
# disco = dcor(METvalue, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(taupairmass, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel(' leading tau pair invariant mass[GeV]')
plt.ylabel('AD score')
plt.title('Density Plot: m_tt vs AD score')

# Add correlation information to the plot
plt.text(0.75, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/taupair.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(taupairmass, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('Leading tau pair invariant mass [GeV]')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim(1e-6,10)
plt.title('L1AD Efficiency vs Leading tau pair invariant mass')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/taupairmass_eff.png')

In [ ]:
correlation, _ = pearsonr(muonpairmass, output_mean_sum_squares)
# disco = dcor(METvalue, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(muonpairmass, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel(' leading muon pair invariant mass[GeV]')
plt.ylabel('AD score')
plt.title('Density Plot: m_mumu vs AD score')

# Add correlation information to the plot
plt.text(0.75, 0.95, f'Correlation: {correlation:.4f}', 
         transform=plt.gca().transAxes, verticalalignment='top', 
         color='black', fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/muonpair.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(muonpairmass, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('Leading muon pair invariant mass [GeV]')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim(1e-6,10)
plt.title('L1AD Efficiency vs Leading muon pair invariant mass')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/muonpairmass_eff.png')

In [ ]:
correlation, _ = pearsonr(totalPT, output_mean_sum_squares)
# disco = dcor(METvalue, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(totalPT, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel(' sum PT')
plt.ylabel('AD score')
plt.title('Density Plot: sum PT vs AD score')

# Add correlation information to the plot
plt.text(0.95, 0.95, f'Correlation: {correlation:.4f}',
        transform=plt.gca().transAxes, 
        verticalalignment='top',
        horizontalalignment='right',
        color='black', 
        fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/sumPT.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(totalPT, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('Sum of PT')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim(1e-6,10)
plt.title('L1AD Efficiency vs Sum of PT')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/totalPT_eff.png')

In [ ]:
columns_of_visible = [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39]
visiblePT = np.sum(test_data[:, columns_of_visible], axis=1)
correlation, _ = pearsonr(visiblePT, output_mean_sum_squares)
# disco = dcor(METvalue, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(visiblePT, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel(' sum PT')
plt.ylabel('AD score')
plt.title('Density Plot: visible sum PT vs AD score')

# Add correlation information to the plot
plt.text(0.95, 0.95, f'Correlation: {correlation:.4f}',
        transform=plt.gca().transAxes, 
        verticalalignment='top',
        horizontalalignment='right',
        color='black', 
        fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/visiblesumPT.png')

In [ ]:
columns_of_visible = [0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39]
visiblePT = np.sum(test_data[:, columns_of_visible], axis=1)
correlation, _ = pearsonr(visiblePT, output_mean_sum_squares)
# disco = dcor(METvalue, output_mean_sum_squares)
# Create the density plot
plt.figure(figsize=(10, 6), dpi=800)
plt.hist2d(visiblePT, output_mean_sum_squares, bins=50, cmap='viridis', 
               norm=LogNorm())
plt.colorbar(label='Count')

plt.xlabel(' sum PT')
plt.ylabel('AD score')
plt.title('Density Plot: visible sum PT vs AD score')

# Add correlation information to the plot
plt.text(0.95, 0.95, f'Correlation: {correlation:.4f}',
        transform=plt.gca().transAxes, 
        verticalalignment='top',
        horizontalalignment='right',
        color='black', 
        fontweight='bold')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Correlation coefficient: {correlation:.4f}")
# print(f"Distance Correlation coefficient: {disco:.4f}")
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/visiblesumPT.png')

In [ ]:
plt.figure(figsize=(10, 6), dpi=800)

# Calculate efficiencies and errors
bins_centers, effs, errs_up, errs_down = calculate_efficiency_with_errors(visiblePT, output_mean_sum_squares, threshold=7.16119766235351562500)

# Plot efficiency points with error bars
plt.errorbar(bins_centers, effs, yerr=[errs_down, errs_up], fmt='o', 
            capsize=3, capthick=1, elinewidth=1, markersize=4,
            label=f'Threshold ≥ {7.16119766235351562500:.5f} (Correspond to 1kHz@L1AD)')

# Add horizontal line at zero
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Customize plot
plt.xlabel('visible Sum of PT')
plt.ylabel('Efficiency')
plt.yscale('log')
plt.ylim(1e-6,10)
plt.title('L1AD Efficiency vs visible Sum of PT')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig('/eos/user/h/hjia/AnomalyDetection/workdir/analysis_notebooks/visiblePT_eff.png')